<a href="https://colab.research.google.com/github/nahum0804/TallerDeProgramacion-Project/blob/main/Reconocer_rostros_III_Etapa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instituto Tecnológico de Costa Rica | TEC
# Carrera de Ingeniería en Computación 
# I Semestre de 2023
# Proyecto Programado (III Fase – Entrevistador asistido por IA) 
# Dr. Abel Méndez Porras
# amendez@itcr.ac.cr
# Porcentaje: 20%

# Entrevistador asistido por IA



El reconocimiento facial y la detección de emociones son tecnologías que se han vuelto cada vez más populares en los últimos años. Estas tecnologías utilizan algoritmos de inteligencia artificial para analizar imágenes y detectar patrones faciales que permiten identificar a las personas y sus emociones.
En cuanto a las oportunidades, el reconocimiento facial puede tener aplicaciones en la seguridad, el marketing y la atención al cliente. Por ejemplo, en la seguridad se pueden utilizar sistemas de reconocimiento facial para identificar a personas en lugares públicos o para controlar el acceso a edificios y sistemas de seguridad. En el marketing, se pueden utilizar para personalizar la publicidad en función de la edad, el género y otros datos demográficos. Y en la atención al cliente, se pueden utilizar para detectar la satisfacción del cliente en tiempo real y responder adecuadamente.

En esta tercera fase del proyecto del curso de Taller de Programación estaremos utilizando los resultados de consultas a modelos de Deep Learning para aplicar los conocimientos adquiridos en clase. Estaremos utilizando los requerimientos desarrollados para la segunda fase del proyecto. Sin embargo, se estarán utilizando widgets para hacer la recolección y mostrar la información para cada requerimiento desarrollado en la segunda fase. Además, se incluyen el manejo de árboles binarios para hacer busqueda de información. 

## Objetivo
Desarrollar una aplicación de complejidad baja que permita poner en práctica los conocimientos adquiridos en los cursos de Introducción a la Programación y Taller de Programación.

## Objetivos Específicos
*  Identificar los requerimientos para resolver un problema específico desde la perspectiva de resolución de problemas, técnicas con listas, diccionarios, clases, objetos, archivos, árboles binarios y widgets.
*  Diseñar una aplicación para un problema específico desde la perspectiva de resolución de problemas, técnicas con listas, diccionarios, clases, objetos, archivos, árboles binarios y widgets.
*  Desarrollar una aplicación para un problema específico desde la perspectiva de resolución de problemas, técnicas con listas, diccionarios, clases, objetos, archivos, árboles binarios y widgets.

## Configuración previa 

### Cargar Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Leer cámara
Tomado de https://colab.research.google.com/drive/1QnC7lV7oVFk5OZCm75fqbLAfD9qBy9bw?usp=sharing#scrollTo=ilLkpcKanPRb



### Importar dependencies

In [2]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

### Convertir el objeto JavaScript un objeto OpenCV image

In [3]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

### Crear nuestro live video stream

In [4]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

### Iniciar streaming video

In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf
from keras.models import model_from_json
from ipywidgets.widgets.widget_output import clear_output
from ipywidgets import Button, TwoByTwoLayout
import ipywidgets as widgets

### Cargar modelos de Deep Learning entrenados

*   Para detectar los rostros se utiliza el modelo de Face Haar Cascade.
*   Para detectar las emociones se utiliza el modelo de VGG.
*   También se utiliza un algoritmo adicional para pasar la información del algortimo de VGG a un formato JSON.

Estos modelos y algoritmos están disponibles en el siguiente enlace [Descargar modelos y algoritmos](https://drive.google.com/drive/folders/1KctA_s25bqqMOUlQuy0lwTm0HkBGVycF?usp=sharing). Usted los debe colocar en su Google Drive y modificar la ruta de acceso. 



In [6]:
#Cargar modelos de Deep Learning entrenados
face_haar_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/projectFiles/haarcascade_frontalface_alt.xml') 
model = model_from_json(open("/content/drive/MyDrive/projectFiles/model.json", "r").read()) 
model.load_weights('/content/drive/MyDrive/projectFiles/model.h5') 

Crear interfaz
https://www.youtube.com/watch?v=oIVmV41uyK8

#Perfiles del Entrevistador Personal

El entrevistador personal es una herramienta para recolectar información de personas que están aplicando a un puesto. En la aplicación existen tres perfiles el entrevistador, el entrevistado y el evaluador. La información de estos perfiles debe ser almacenada en clases y usando herencia. Debe existir una clase "padre" que tiene propiedades y métodos que comparten todos los perfiles. Para cada perfil debe existir una clase subordinada que hereda las propiedades de la clase "padre", además, tiene sus propias propiedades y métodos.

In [38]:
#Clase Padre
from datetime import date, datetime

class Person():

    def __init__(self, name, id, gender, birthDay, nacionality, address):
        """
        Initializes a Person object with user-provided information.

        The user is prompted to enter their ID, name, gender, birth date, nationality, and address.
        These details are stored as instance variables.

        Parameters:
        None

        Returns:
        None
        """

        self.id = id
        self.name = name
        self.gender = gender
        self.birthDay = birthDay
        self.nacionality = nacionality
        self.address = address

    def calcularEdad(self):
      """
      Calculates the age of the person based on their birth date.

      The birth date is retrieved from the instance variable 'birthDay',
      and the current date is used to calculate the age. The age is
      returned as an integer value.

      Parameters:
      None

      Returns:
      int: The age of the person.
      """
      today = date.today()
      fecha_nacimiento_str = self.birthDay
      fecha_nacimiento = datetime.strptime(fecha_nacimiento_str, "%d-%m-%Y").date()
      edad = today.year - fecha_nacimiento.year - ((today.month, today.day) < (fecha_nacimiento.month, fecha_nacimiento.day))
      return edad

## Recolectar información del entrevistador


El entrevistador es la persona encargada de realizar las entrevistas. 

Información requerida del entrevistador (Toda la información requerida debe ser almacenada en la clase Entrevistador. Cada vez que se registra un entrevistador se debe crear una instancia de la clase Entrevistador para guardar toda la información del entrevistador). 
*   Identificación  
*   Nombre 
*   Fecha de nacimiento
*   Nacionalidad
*   Números de teléfono y celular (un diccionario)
*   Emails (una lista con uno o varios emails)
*   Dirección

Usted debe incluir al menos tres propiedades únicas al entrevistador.
Cada vez que se crea una instancia de Entrevistador, la instancia debe ser almacenada en un archivo binario llamado Archivo_Entrevistador.

In [39]:
#Su código
class Interviewer(Person):
    def __init__(self, name, id, gender, birthDay, nacionality, address, civilStatus, cellphoneData, emails, ocupattion, yearsOfExperience, trained):
        """
        Initializes an Interviewer object with user-provided information.

        Inherits from the Person class and adds additional attributes specific to an interviewer,
        including 'trained', 'occupation', 'yearsOfExperience', 'cellphones', and 'emails'.

        Parameters:
        None

        Returns:
        None
        """
        super().__init__(name, id, gender, birthDay, nacionality, address)
        self.trained = trained                                        # 1 - New Attribute
        self.ocupattion = ocupattion                                  # 2 - New Attribute
        self.yearsOfExperience = yearsOfExperience                    # 3 - New Attribute
        self.cellphones = cellphoneData
        self.emails = emails


Function Interviewer Getting Data

In [40]:
def interviewerGettingData(b):
  clear_output()

  label = widgets.Label(value="Interviewer Menu")
  buttonAddInterviewerData = widgets.Button(description="Add New")
  buttonShowInterviewerData = widgets.Button(description="Show Data")
  buttonBack = widgets.Button(description='Back', disabled=False, button_style='', tooltip='Click me', icon='check')
  buttonBack.style.button_color = '#1CA9C9'

  display(label)
  display(buttonAddInterviewerData)
  display(buttonShowInterviewerData)
  display(buttonBack)

  buttonAddInterviewerData.on_click(addInterviewer)
  buttonShowInterviewerData.on_click(showInterviewerData)
  buttonBack.on_click(main)

In [41]:
def addInterviewer(b):
  outputCellphone = widgets.Output()
  outputEmail = widgets.Output()

  email_inputs = list() 
  cellphone_inputs = dict()

  name = widgets.Text(
    value='Name here!', 
    placeholder='Type your name',  
    description='Name:',
    disabled=False         
  )

  id = widgets.Text(
    value='ID here!', 
    placeholder='Type your ID',  
    description='ID:',
    disabled=False         
  )

  birthDay =  widgets.Text(
    value='DD-MM-YYYY', 
    placeholder='DD-MM-YYYY',  
    description='BirthDay:',
    disabled=False         
  )

  gender = widgets.Dropdown(
    options = [ 'Male','Female'],
    value='Female',
    description='Gender:',
    disabled=False,
  )
  

  nacionality = widgets.Dropdown(
    options = [ 'Argentina','Belice','Bolivia','Brasil','Canadá','Chile','Colombia','Costa Rica','Cuba','Dominica','Ecuador','El Salvador','Estados Unidos','Guatemala','Haití','Honduras','Jamaica','México','Nicaragua','Panamá','Paraguay','Perú','República Dominicana','Trinidad y Tobago','Uruguay','Venezuela'],
    value='Costa Rica',
    description='Nacionality:',
    disabled=False,
  )
  
  address = widgets.Textarea(
    value='Location here',
    placeholder='Type your location',
    description='Location:',
    disabled=False
   )
  
  civilStatus = widgets.Dropdown(
    options=['Married', 'Divorced', 'Single', 'Other'],
    value='Single',
    description='Civil Status:',
    disabled=False,
  )
  
#  ocupattion, yearsOfExperience
  ocupattion = widgets.Textarea(
    value='Ocupattion',
    placeholder='Whats your ocupattion?',
    description='Ocupattion:',
    disabled=False
   )
  
  yearsOfExperience = widgets.BoundedIntText(
      value=1,              #Inicial value
      min=0,                #Min Value
      max=70,               #Max Value
      step=1,               #Step
      description='Experience (Years):',
      style={'description_width':'initial'},
      disabled=False
  )

  trained = widgets.ToggleButtons(
    options=["True", "False"],
    description='Trained:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['The interviewer is trained', 'The interviewer is not trained'],
  )
  
  cellphoneData = widgets.BoundedIntText(
      value=1,              #Inicial value
      min=0,                #Min Value
      max=2,               #Max Value
      step=1,               #Step
      description='Quantity cellphone numbers:',
      style={'description_width':'initial'},
      disabled=False
      )

  emailList = widgets.BoundedIntText(
      value=0,              #Inicial value
      min=0,                #Min Value
      max=2,               #Max Value
      step=1,               #Step
      description='Quantity emails:',
      style={'description_width':'initial'},
      disabled=False
      )
    
  buttonCellphones = widgets.Button(
    description='Load cellphone Input',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  buttonEmails = widgets.Button(
    description='Load email Inputs',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  clear_output()
  display(name, id, gender, birthDay, nacionality, address, civilStatus, ocupattion, yearsOfExperience, trained, cellphoneData, buttonCellphones, emailList, buttonEmails)

  button = widgets.Button(
    description='Camera',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )



  #----Showing quantity inputs for cellphone-----------
  def showCellphoneInputs(b, cellphone_inputs):
    titulo_widget = widgets.Label(value="Cellphone Inputs")
    with outputCellphone:
      clear_output()
      if cellphoneData.value == 0:
        pass
      elif cellphoneData.value == 1:
        display(titulo_widget)

        cellphone = widgets.Text(
          value=f'Cellphone number 1', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone_inputs["Cellphone 1"] = cellphone
        display(cellphone)

      elif cellphoneData.value == 2:
        display(titulo_widget)

        cellphone = widgets.Text(
          value=f'Cellphone number 1', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone2 = widgets.Text(
          value=f'Cellphone 2', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 2:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone_inputs["Cellphone 1"] = cellphone
        cellphone_inputs["Cellphone 2"] = cellphone2
        display(widgets.VBox([cellphone, cellphone2]))  
          


  buttonCellphones.on_click(lambda b: showCellphoneInputs(b, cellphone_inputs))
  #----------------------------------------------------
  display(outputCellphone)

  #--Get cellphone data-------------------
  def obtenerValoresCellphones():
    valores_cellphones = {key: cellphone.value for key, cellphone in cellphone_inputs.items()}
    return valores_cellphones
  #-----------------------------------


  #----Showing quantity inputs for cellphone-----------
  def showEmailsInputs(b, email_inputs):
    titulo_widget = widgets.Label(value="Emails Inputs")
    with outputEmail:
      clear_output()
      if emailList.value == 0:
        pass
      elif emailList.value == 1:
        display(titulo_widget)

        email = widgets.Text(
          value=f'Email 1', 
          placeholder='Type your email',  
          description=f'Email 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        email_inputs.append(email)
        display(email)



      elif emailList.value == 2:
        display(titulo_widget)

        email = widgets.Text(
          value=f'Email 1', 
          placeholder='Type your email',  
          description=f'Email 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        
        email2 = widgets.Text(
          value=f'Email 2', 
          placeholder='Type your email',  
          description=f'Email 2:',
          style={'description_width':'initial'},
          disabled=False         
          )
        email_inputs.extend([email, email2])
        display(widgets.VBox([email, email2]))     

  buttonEmails.on_click(lambda b: showEmailsInputs(b, email_inputs))
  #----------------------------------------------------
  display(outputEmail)


  #--Get email data-------------------
  def obtenerValoresEmails():
    valores_emails = [email.value for email in email_inputs]
    return valores_emails
  #-----------------------------------


  #----Save Data----------------------
  def guardar(b):
    newInterviewer = Interviewer(name.value, id.value, gender.value, birthDay.value, nacionality.value, address.value, civilStatus.value, {}, [], ocupattion.value, yearsOfExperience.value, trained.value)
    newInterviewer.emails = obtenerValoresEmails()
    newInterviewer.cellphones = obtenerValoresCellphones()



    with open("/content/drive/MyDrive/projectFiles/IIIStage-Project/Interviewer_SaveFile.bin", "wb") as f:
      pickle.dump(newInterviewer, f)
    
   

    name.value = ""
    id.value = ""
    gender.value = "Female"
    birthDay.value = "DD-MM-YYYY"
    nacionality.value = "Costa Rica"
    address.value = "Location here!" 
    cellphoneData.value = 0
    emailList.value = 0
  
  def ejecutarFunciones(b):
    guardar(b)
    main(b)
  
  button.on_click(ejecutarFunciones)
  #------------------------------------
  display(button)

In [42]:
def showInterviewerData(b):
   clear_output()

   buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
   buttonBack.style.button_color = '#1CA9C9'

   try:
       with open("/content/drive/MyDrive/projectFiles/IIIStage-Project/Interviewer_SaveFile.bin", "rb") as f:
            savedInterviewer = pickle.load(f)

            label = widgets.Label(value="Interviewer Data:")
            name = widgets.Text(value=savedInterviewer.name, description='Name:', disabled=True)
            id = widgets.Text(value=savedInterviewer.id, description='ID:', disabled=True)
            birthDay = widgets.Text(value=savedInterviewer.birthDay, description='BirthDay:', disabled=True)
            gender = widgets.Text(value=savedInterviewer.gender, description='Gender:', disabled=True)
            nacionality = widgets.Text(value=savedInterviewer.nacionality, description='Nacionality:', disabled=True)
            address = widgets.Textarea(value=savedInterviewer.address, description='Location:', disabled=True)
            ocupattion = widgets.Textarea(value=savedInterviewer.ocupattion, description='Ocupattion:', disabled=True)
            yearsOfExperience = widgets.Textarea(value=str(savedInterviewer.yearsOfExperience), description='Experience (Years):', disabled=True)
            trained = widgets.Text(value=savedInterviewer.trained, description='Trained:', disabled=True)

            cellphoneData = widgets.BoundedIntText(
                value=len(savedInterviewer.cellphones),
                min=0,
                max=2,
                step=1,
                description='Quantity cellphone numbers:',
                style={'description_width': 'initial'},
                disabled=True
            )

            emailList = widgets.BoundedIntText(
                value=len(savedInterviewer.emails),
                min=0,
                max=2,
                step=1,
                description='Quantity emails:',
                style={'description_width': 'initial'},
                disabled=True
            )

            display(label)
            display(name)
            display(id)
            display(birthDay)
            display(gender)
            display(nacionality)
            display(address)
            display(ocupattion)
            display(yearsOfExperience)
            display(trained)
            display(cellphoneData)
            display(emailList)

            # Imprimir correos electrónicos
            if savedInterviewer.emails:
                emailLabel = widgets.Label(value="Emails:")
                display(emailLabel)
                for email in savedInterviewer.emails:
                    emailWidget = widgets.Text(value=email, disabled=True)
                    display(emailWidget)

            # Imprimir números de teléfono
            lista = list(savedInterviewer.cellphones)
            # print(lista)
            phoneLabel = widgets.Label(value="Phone Numbers:")
            display(phoneLabel)
            for i in lista:
               phone_number = savedInterviewer.cellphones[i]
               phoneWidget = widgets.Text(value=phone_number, disabled=True)
               display(phoneWidget)
     
            display(buttonBack)
            buttonBack.on_click(main)

   except FileNotFoundError:
        print("Error 404. Interviewer Data not found. :(")
        display(buttonBack)
        buttonBack.on_click(main)


## Recolectar información del evaluador

El evaluador en la persona encargada de revisar la información obtenida de cada entrevista. 

Información requerida del evaluador (Toda la información requerida debe ser almacenada en la clase Evaluador. Cada vez que se registra un evaluador se debe crear una instancia de la clase Evaluador para guardar toda la información del evaluador). 
*   Identificación  
*   Nombre 
*   Fecha de nacimiento
*   Nacionalidad
*   Números de teléfono y celular (un diccionario)
*   Emails (una lista con uno o varios emails)
*   Dirección

Usted debe incluir al menos tres propiedades únicas al evaluador.
Cada vez que se crea una instancia de Evaluador, la instancia debe ser almacenada en un archivo binario llamado Archivo_Evaluador.

In [43]:
#Su código
class Evaluator(Person):
    def __init__(self, name, id, gender, birthDay, nacionality, address, civilStatus, career, salary, level, cellphoneData, emails):
        """
        Initializes an Evaluator object with user-provided information.

        Inherits from the Person class and adds additional attributes specific to an evaluator,
        including 'career', 'salary', 'level', 'cellphones', and 'emails'.

        Parameters:
        None

        Returns:
        None
        """
        super().__init__(name, id, gender, birthDay, nacionality, address)
        self.career = career # 1 - New Attribute
        self.salary = salary                      # 2 - New Attribute
        #print("Levels: \n * Hard \n * Intermediate \n * Soft")
        self.level = level            # 3 - New Attribute
        self.cellphones = cellphoneData
        self.emails = emails

Funcion del formulario

In [44]:
def evaluatorSave(b):
   clear_output()
   label = widgets.Label(value="Evaluator Menu")
   buttonAddEvaluatorData = widgets.Button(description="Add New")
   buttonShowEvaluatorData = widgets.Button(description="Show Data")
   buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
   buttonBack.style.button_color = '#1CA9C9'

   display(label)
   display(buttonAddEvaluatorData)
   display(buttonShowEvaluatorData)
   display(buttonBack)

   buttonAddEvaluatorData.on_click(addEvaluator)
   buttonShowEvaluatorData.on_click(showEvaluatorData)
   buttonBack.on_click(main)

In [45]:
def addEvaluator(b):
  outputCellphone = widgets.Output()
  outputEmail = widgets.Output()

  email_inputs = list() 
  cellphone_inputs = dict()

  name = widgets.Text(
    value='Name here!', 
    placeholder='Type your name',  
    description='Name:',
    disabled=False         
  )

  id = widgets.Text(
    value='ID here!', 
    placeholder='Type your ID',  
    description='ID:',
    disabled=False         
  )

  birthDay =  widgets.Text(
    value='DD-MM-YYYY', 
    placeholder='DD-MM-YYYY',  
    description='BirthDay:',
    disabled=False         
  )

  gender = widgets.Dropdown(
    options = [ 'Male','Female'],
    value='Female',
    description='Gender:',
    disabled=False,
  )
  

  nacionality = widgets.Dropdown(
    options = [ 'Argentina','Belice','Bolivia','Brasil','Canadá','Chile','Colombia','Costa Rica','Cuba','Dominica','Ecuador','El Salvador','Estados Unidos','Guatemala','Haití','Honduras','Jamaica','México','Nicaragua','Panamá','Paraguay','Perú','República Dominicana','Trinidad y Tobago','Uruguay','Venezuela'],
    value='Costa Rica',
    description='Nacionality:',
    disabled=False,
  )
  
  address = widgets.Textarea(
    value='Location here',
    placeholder='Type your location',
    description='Location:',
    disabled=False
   )
  
  civilStatus = widgets.Dropdown(
    options=['Married', 'Divorced', 'Single', 'Other'],
    value='Single',
    description='Civil Status:',
    disabled=False,
  )
  
  career = widgets.Textarea(
    value='Career',
    placeholder='Career',
    description='Career:',
    disabled=False
  )
  
  salary = widgets.Textarea(
    value='Salary',
    placeholder='Salary',
    description='Salary:',
    disabled=False
  )

  level = widgets.Dropdown(
    options = [ 'Hard','Intermediate','Soft'],
    value='Intermediate',
    description='Level:',
    disabled=False,
  )

  
  cellphoneData = widgets.BoundedIntText(
      value=1,              #Inicial value
      min=0,                #Min Value
      max=2,               #Max Value
      step=1,               #Step
      description='Quantity cellphone numbers:',
      style={'description_width':'initial'},
      disabled=False
      )

  emailList = widgets.BoundedIntText(
      value=0,              #Inicial value
      min=0,                #Min Value
      max=2,               #Max Value
      step=1,               #Step
      description='Quantity emails:',
      style={'description_width':'initial'},
      disabled=False
      )
    
  buttonCellphones = widgets.Button(
    description='Load cellphone Input',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  buttonEmails = widgets.Button(
    description='Load email Inputs',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )


  clear_output()
  display(name, id, gender, birthDay, nacionality, address, civilStatus, career, salary, level, cellphoneData, buttonCellphones, emailList, buttonEmails)

  button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  #----Showing quantity inputs for cellphone-----------
  def showCellphoneInputs(b, cellphone_inputs):
    titulo_widget = widgets.Label(value="Cellphone Inputs")
    with outputCellphone:
      clear_output()
      if cellphoneData.value == 0:
        pass
      elif cellphoneData.value == 1:
        display(titulo_widget)

        cellphone = widgets.Text(
          value=f'Cellphone number 1', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone_inputs["Cellphone 1"] = cellphone
        display(cellphone)

      elif cellphoneData.value == 2:
        display(titulo_widget)

        cellphone = widgets.Text(
          value=f'Cellphone number 1', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone2 = widgets.Text(
          value=f'Cellphone 2', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 2:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone_inputs["Cellphone 1"] = cellphone
        cellphone_inputs["Cellphone 2"] = cellphone2
        display(widgets.VBox([cellphone, cellphone2]))  
          


  buttonCellphones.on_click(lambda b: showCellphoneInputs(b, cellphone_inputs))
  #----------------------------------------------------
  display(outputCellphone)

  #--Get cellphone data-------------------
  def obtenerValoresCellphones():
    valores_cellphones = {key: cellphone.value for key, cellphone in cellphone_inputs.items()}
    return valores_cellphones
  #-----------------------------------


  #----Showing quantity inputs for cellphone-----------
  def showEmailsInputs(b, email_inputs):
    titulo_widget = widgets.Label(value="Emails Inputs")
    with outputEmail:
      clear_output()
      if emailList.value == 0:
        pass
      elif emailList.value == 1:
        display(titulo_widget)

        email = widgets.Text(
          value=f'Email 1', 
          placeholder='Type your email',  
          description=f'Email 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        email_inputs.append(email)
        display(email)



      elif emailList.value == 2:
        display(titulo_widget)

        email = widgets.Text(
          value=f'Email 1', 
          placeholder='Type your email',  
          description=f'Email 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        
        email2 = widgets.Text(
          value=f'Email 2', 
          placeholder='Type your email',  
          description=f'Email 2:',
          style={'description_width':'initial'},
          disabled=False         
          )
        email_inputs.extend([email, email2])
        display(widgets.VBox([email, email2]))     

  buttonEmails.on_click(lambda b: showEmailsInputs(b, email_inputs))
  #----------------------------------------------------
  display(outputEmail)


  #--Get email data-------------------
  def obtenerValoresEmails():
    valores_emails = [email.value for email in email_inputs]
    return valores_emails
  #-----------------------------------


  #----Save Data----------------------
  def guardar(b):
    newEvaluator = Evaluator(name.value, id.value, gender.value, birthDay.value, nacionality.value, address.value, civilStatus.value, career.value, salary.value, level.value, {}, [])
    newEvaluator.emails = obtenerValoresEmails()
    newEvaluator.cellphones = obtenerValoresCellphones()



    with open("/content/drive/MyDrive/projectFiles/IIIStage-Project/Evaluador_SaveFile.bin", "wb") as f:
      pickle.dump(newEvaluator, f)
    
   

    name.value = ""
    id.value = ""
    gender.value = "Female"
    birthDay.value = "DD-MM-YYYY"
    nacionality.value = "Costa Rica"
    address.value = "Location here!" 
    cellphoneData.value = 0
    emailList.value = 0
  
  def ejecutarFunciones(b):
    guardar(b)
    main(b)

  button.on_click(ejecutarFunciones)
  #------------------------------------
  display(button)

In [46]:
def showEvaluatorData(b):
    clear_output()

    buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
    buttonBack.style.button_color = '#1CA9C9'

    try:
        with open("/content/drive/MyDrive/projectFiles/IIIStage-Project/Evaluador_SaveFile.bin", "rb") as f:
            savedEvaluator = pickle.load(f)

            label = widgets.Label(value="Evaluator Data:")
            name = widgets.Text(value=savedEvaluator.name, description='Name:', disabled=True)
            id = widgets.Text(value=savedEvaluator.id, description='ID:', disabled=True)
            birthDay = widgets.Text(value=savedEvaluator.birthDay, description='BirthDay:', disabled=True)
            gender = widgets.Text(value=savedEvaluator.gender, description='Gender:', disabled=True)
            nacionality = widgets.Text(value=savedEvaluator.nacionality, description='Nacionality:', disabled=True)
            address = widgets.Textarea(value=savedEvaluator.address, description='Location:', disabled=True)
            career = widgets.Textarea(value=savedEvaluator.career, description='Career:', disabled=True)
            salary = widgets.Textarea(value=savedEvaluator.salary, description='Salary:', disabled=True)
            level = widgets.Text(value=savedEvaluator.level, description='Level:', disabled=True)

            cellphoneData = widgets.BoundedIntText(
                value=len(savedEvaluator.cellphones),
                min=0,
                max=2,
                step=1,
                description='Quantity cellphone numbers:',
                style={'description_width': 'initial'},
                disabled=True
            )

            emailList = widgets.BoundedIntText(
                value=len(savedEvaluator.emails),
                min=0,
                max=2,
                step=1,
                description='Quantity emails:',
                style={'description_width': 'initial'},
                disabled=True
            )

            display(label)
            display(name)
            display(id)
            display(birthDay)
            display(gender)
            display(nacionality)
            display(address)
            display(career)
            display(salary)
            display(level)
            display(cellphoneData)
            display(emailList)

            # Imprimir correos electrónicos
            if savedEvaluator.emails:
                emailLabel = widgets.Label(value="Emails:")
                display(emailLabel)
                for email in savedEvaluator.emails:
                    emailWidget = widgets.Text(value=email, disabled=True)
                    display(emailWidget)

            # Imprimir números de teléfono
            lista = list(savedEvaluator.cellphones)
            # print(lista)
            phoneLabel = widgets.Label(value="Phone Numbers:")
            display(phoneLabel)
            for i in lista:
               phone_number = savedEvaluator.cellphones[i]
               phoneWidget = widgets.Text(value=phone_number, disabled=True)
               display(phoneWidget)
     

            # if savedEvaluator.cellphones:
            #     phoneLabel = widgets.Label(value="Phone Numbers:")
            #     display(phoneLabel)
            #     for phone in savedEvaluator.cellphones:
            #         phoneWidget = widgets.Text(value=phone, disabled=True)
            #         display(phoneWidget)

            display(buttonBack)
            buttonBack.on_click(main)

    except FileNotFoundError:
        print("Error 404. Evaluator Data not found. :(")
        display(buttonBack)
        buttonBack.on_click(main)


## Recolectar información del entrevistado

El entrevistado es la persona que está interesada en el puesto de trabajo.

Información requerida al entrevistar una persona (Toda la información requerida debe ser almacenada en la clase Entrevistado. Cada vez que se registra un entrevistado se debe crear una instancia de la clase Entrevistado para guardar toda la información del entrevistado).
*   Identificación  
*   Nombre 
*   Fecha de nacimiento
*   Nacionalidad
*   Números de teléfono y celular (un diccionario)
*   Emails (una lista con uno o varios emails)
*   Dirección
*   Estado civil
*   Cantidad de hijos indicando el género, la fecha de nacimiento(un diccionario) 
*   Entrevistas (cada persona puede ser entrevistada cero o más veces). Las entrevistas se realizan en tiempo real utilizando una camára integrada a la computadora o externa (el profesor facilita el código para activar la camára, capturar el video y capturar los frames cada cierto tiempo). Se debe crear una lista con sublistas donde cada sublista tiene las imagenes capturas y las emociones reportadas durante la entrevista. Además, debe guardar coordenadas donde se encuentra el rostro en la imagen. Para cada entrevista, el entrevistado debe contar historias cortas sobre los siguientes 4 tópicos:


  >*   Una experiencia de sorpresa o asombro que ha tenido que vivir.
  >*   Una experiencia de tristeza que ha tenido que vivir.
  >*   Una experiencia de enojo o frustración que ha tenido que vivir.
  >*   Una experiencia de felicidad que ha tenido que vivir.

Un aspecto importante es que cada vez que se va a realizar una entrevista se debe indicar quién es el entrevistador.

Usted debe incluir al menos tres propiedades únicas al perfil de entrevistado. Cada vez que se crea una instancia de Entrevistado, la instancia debe ser almacenada en un archivo binario llamado Archivo_Entrevistado.

In [47]:
#Su código
#Su código
class Interviewed(Person):
    def __init__(self, name, id, gender, birthDay, nacionality, address, leguageNative, accepted, civilStatus, cellphoneData, emails):
        """
        Initializes a Person object with user-provided information.

        The user is prompted to enter their ID, name, gender, birth date, nationality, and address.
        These details are stored as instance variables.

        Parameters:
        None

        Returns:
        None
        """
        super().__init__(name, id, gender, birthDay, nacionality, address)
        self.technicalSkills = []                          # 1 - New Attribute
        self.softSkills = []                               # 2 - New Attribute
        self.lenguageNative = leguageNative                # 3 - New Attribute
        self.accepted = accepted
        self.civilStatus = civilStatus
        self.cellphones = cellphoneData
        #i = 1
        self.cellphonCant = cellphoneData
        self.emails = emails
        self.photoData = []
        self.childrenData = {}

###Recolectar datos de la entrevista

In [48]:
from ipywidgets.widgets.widget_output import clear_output
import ipywidgets as widgets
import pickle

#Original

def asignInterviewed(b):

  outputCellphone = widgets.Output()
  outputEmail = widgets.Output()
  outputChildren = widgets.Output()

  email_inputs = list() 
  cellphone_inputs = dict()
  children_dict = dict()


  name = widgets.Text(
    value='Name here!', 
    placeholder='Type your name',  
    description='Name:',
    disabled=False         
  )

  id = widgets.Text(
    value='ID here!', 
    placeholder='Type your ID',  
    description='ID:',
    disabled=False         
  )

  birthDay =  widgets.Text(
    value='DD-MM-YYYY', 
    placeholder='DD-MM-YYYY',  
    description='BirthDay:',
    disabled=False         
  )

  gender = widgets.Dropdown(
    options = [ 'Male','Female'],
    value='Female',
    description='Gender:',
    disabled=False,
  )
  

  nacionality = widgets.Dropdown(
    options = [ 'Argentina','Belice','Bolivia','Brasil','Canadá','Chile','Colombia','Costa Rica','Cuba','Dominica','Ecuador','El Salvador','Estados Unidos','Guatemala','Haití','Honduras','Jamaica','México','Nicaragua','Panamá','Paraguay','Perú','República Dominicana','Trinidad y Tobago','Uruguay','Venezuela'],
    value='Costa Rica',
    description='Nacionality:',
    disabled=False,
  )
  
  address = widgets.Textarea(
    value='Location here',
    placeholder='Type your location',
    description='Location:',
    disabled=False
   )
  
  nativeLenguage = widgets.Text(
    value='Spanish', 
    placeholder='Type your Native Lenguage',  
    description='Native Lenguage:',
    style={'description_width':'initial'},
    disabled=False         
  )
  civilStatus = widgets.Dropdown(
    options=['Married', 'Divorced', 'Single', 'Other'],
    value='Single',
    description='Civil Status:',
    disabled=False,
    )
  
  state = widgets.ToggleButtons(
    options=['Accepted', 'Denied'],
    description='State: ',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    #     icons=['check'] * 3
    ) 
  
  cellphoneData = widgets.BoundedIntText(
      value=1,              #Inicial value
      min=0,                #Min Value
      max=2,               #Max Value
      step=1,               #Step
      description='Quantity cellphone numbers:',
      style={'description_width':'initial'},
      disabled=False
      )

  emailList = widgets.BoundedIntText(
      value=0,              #Inicial value
      min=0,                #Min Value
      max=2,               #Max Value
      step=1,               #Step
      description='Quantity emails:',
      style={'description_width':'initial'},
      disabled=False
      )
  childrenQuantity = widgets.BoundedIntText(
      value=0,              #Inicial value
      min=0,                #Min Value
      max=14,               #Max Value
      step=1,               #Step
      description='Quantity children:',
      style={'description_width':'initial'},
      disabled=False
      )
    
  buttonCellphones = widgets.Button(
    description='Load cellphone Input',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  buttonEmails = widgets.Button(
    description='Load email Inputs',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  buttonChildren = widgets.Button(
    description='Load Children Inputs',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  clear_output()
  display(name, id, gender, birthDay, nacionality, address, nativeLenguage, civilStatus, cellphoneData, buttonCellphones, emailList, buttonEmails, childrenQuantity, buttonChildren)

  button = widgets.Button(
    description='Camera',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  button.style.button_color = '#61FF33'

  buttonH = widgets.Button(
    description='Home',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  buttonH.style.button_color = '#1CA9C9'

  #----Showing quantity inputs for cellphone-----------
  def showCellphoneInputs(b, cellphone_inputs):
    titulo_widget = widgets.Label(value="Cellphone Inputs")
    with outputCellphone:
      clear_output()
      if cellphoneData.value == 0:
        pass
      elif cellphoneData.value == 1:
        display(titulo_widget)

        cellphone = widgets.Text(
          value=f'Cellphone number 1', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone_inputs["Cellphone 1"] = cellphone
        display(cellphone)

      elif cellphoneData.value == 2:
        display(titulo_widget)

        cellphone = widgets.Text(
          value=f'Cellphone number 1', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone2 = widgets.Text(
          value=f'Cellphone 2', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 2:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone_inputs["Cellphone 1"] = cellphone
        cellphone_inputs["Cellphone 2"] = cellphone2
        display(widgets.VBox([cellphone, cellphone2]))  
          


  buttonCellphones.on_click(lambda b: showCellphoneInputs(b, cellphone_inputs))
  #----------------------------------------------------
  display(outputCellphone)

  #--Get cellphone data-------------------
  def obtenerValoresCellphones():
    valores_cellphones = {key: cellphone.value for key, cellphone in cellphone_inputs.items()}
    return valores_cellphones
  #-----------------------------------


  #----Showing quantity inputs for cellphone-----------
  def showEmailsInputs(b, email_inputs):
    titulo_widget = widgets.Label(value="Emails Inputs")
    with outputEmail:
      clear_output()
      if emailList.value == 0:
        pass
      elif emailList.value == 1:
        display(titulo_widget)

        email = widgets.Text(
          value=f'Email 1', 
          placeholder='Type your email',  
          description=f'Email 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        email_inputs.append(email)
        display(email)



      elif emailList.value == 2:
        display(titulo_widget)

        email = widgets.Text(
          value=f'Email 1', 
          placeholder='Type your email',  
          description=f'Email 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        
        email2 = widgets.Text(
          value=f'Email 2', 
          placeholder='Type your email',  
          description=f'Email 2:',
          style={'description_width':'initial'},
          disabled=False         
          )
        email_inputs.extend([email, email2])
        display(widgets.VBox([email, email2]))     

  buttonEmails.on_click(lambda b: showEmailsInputs(b, email_inputs))
  #----------------------------------------------------
  display(outputEmail)


  #--Get email data-------------------
  def obtenerValoresEmails():
    valores_emails = [email.value for email in email_inputs]
    return valores_emails
  #-----------------------------------


#----Show Quantity Children inputs----------------
  def showChildrenInputs(b, children_dict):
    with outputChildren:
        clear_output()
        titulo_widget = widgets.Label(value="Children Inputs")
        display(titulo_widget)

        quantity = childrenQuantity.value

        child_widgets = []

        for i in range(quantity):
            name = widgets.Text(
                value='Name here!', 
                placeholder='Type your name',  
                description='Name:',
                disabled=False         
            )

            gender = widgets.Dropdown(
                options=['Male', 'Female'],
                value='Female',
                description='Gender:',
                disabled=False,
            )

            birthDay = widgets.Text(
                value='DD-MM-YYYY',
                placeholder='DD-MM-YYYY',
                description='BirthDay:',
                disabled=False
            )

            children_dict[str(i+1)] = {
                'name': name,
                'gender': gender,
                'birthDay': birthDay
            }

            if i > 0:
                titulo_widget_child = widgets.Label(value=f"Child {i+1}:")
                child_widgets.append(titulo_widget_child)

            child_widgets.extend([name, gender, birthDay])

        display(widgets.VBox(child_widgets))
    

  def getChildrenData():
    data_dict = {}
    
    for key, value in children_dict.items():
        child_data = {}
        child_data['name'] = value['name'].value
        child_data['gender'] = value['gender'].value
        child_data['birthDay'] = value['birthDay'].value
        data_dict[key] = child_data
    
    return data_dict

  buttonChildren.on_click(lambda b: showChildrenInputs(b, children_dict))
#-------------------------------------------------
  display(outputChildren)


  #----Save Data----------------------
  def guardar(b):
    newInterviewed = Interviewed(name.value, id.value, gender.value, birthDay.value, nacionality.value, address.value, nativeLenguage.value, state.value, civilStatus.value, {}, None)
    newInterviewed.emails = obtenerValoresEmails()
    newInterviewed.cellphones = obtenerValoresCellphones()
    newInterviewed.childrenData = getChildrenData()
    photoData = process_video_stream(name.value)
    newInterviewed.photoData = photoData 


    with open("/content/drive/MyDrive/projectFiles/IIIS-Testing/testingData.bin", "ab") as f:
      pickle.dump(newInterviewed, f)
    
   

    name.value = ""
    id.value = ""
    gender.value = "Female"
    birthDay.value = "DD-MM-YYYY"
    nacionality.value = "Costa Rica"
    address.value = "Location here!" 
    cellphoneData.value = 0
    emailList.value = 0
  
  def ejecutarFunciones(b):
    guardar(b)
    main(b)
  
  def home(b):
    main(b)

  button.on_click(ejecutarFunciones)
  buttonH.on_click(home)

  button.on_click(ejecutarFunciones)
  #------------------------------------
  display(widgets.HBox([button, buttonH]))

#asignInterviewed()

### Recolectar información emociones
Este código se encarga de ejecutar la captura de video de la cámara, detectar los rostros en las imágenes y reconocer las emociones presentes en los rostros. Cuando se hace la entrevista es en este código donde usted de obtener la información para crear las listas con la información solicitada.

In [49]:
from tensorflow.python.ops.math_ops import sign
import time

def process_video_stream(objectRname):

    photoData = list()

    # start streaming video from webcam
    video_stream()
    # label for video
    label_html = 'Capturing...'
    # initialze bounding box to empty
    bbox = ''
    count = 0 
    j = 5

    while True:
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            break

        # convert JS response to OpenCV Image
        img = js_to_image(js_reply["img"])
        img_copy = img

        # create transparent overlay for bounding box
        bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

        if count == j: # El tiempo para capturar el frame y procesarlo
            # Identify unrepeatable photo
            var = time.time()
            # Este sería el path de la imagen guardada
            image_path = '/content/drive/MyDrive/projectFiles/IIIS-Testing/' + objectRname + str(var) + '.jpg'
            # se guarda el fotograma con el "nombre" más el número del contador i
            cv2.imwrite(image_path , img) 
            
            # Detectar los rostros usando Face Haar Cascade
            gray_image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            # En faces_detected se encuentran los rotros detectados
            faces_detected = face_haar_cascade.detectMultiScale(gray_image, 1.32, 5)
            # Imprimir ubicación de los rostros en la imagen
            #print("Ubicación del rostro:", faces_detected)
            im2Display = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imTemp = im2Display.copy()

            # Hacer un rectángulo en cada rostro detectado en la imagen
            for (x, y, w, h) in faces_detected:
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), thickness=7)

                roi_gray = gray_image[y:y + w, x:x + h]
                roi_gray = cv2.resize(roi_gray, (48, 48))

                # Processes the image and adjust it to pass it to the model
                image_pixels = tf.keras.preprocessing.image.img_to_array(roi_gray)
               
                image_pixels = np.expand_dims(image_pixels, axis=0)
                image_pixels /= 255

                # Obtener las emociones para cada rostro detectado en la imagen
                # Get the prediction of the model
                predictions = model.predict(image_pixels)
                # Mostrar todas las emociones detectadas
                #print("Predicciones: ", predictions)
                # Mostrar la emoción más significativa (con el valor más alto)
                max_index = np.argmax(predictions[0])
                #print(max_index, predictions[0])
                # El orden en que vienen las emociones
                emotion_detection = ('angry', 'disgust', 'fear', 'happyness', 'sadness', 'surprise', 'neutral')

                emotion_prediction = emotion_detection[max_index]
                # Crear un diccionario para almacenar la emoción y el valor para la emoción
                frame_dic = dict(
                    angry=predictions[0][0],
                    disgust=predictions[0][1],
                    fear=predictions[0][2],            
                    happiness=predictions[0][3],
                    sadness=predictions[0][4],
                    surprise=predictions[0][5],
                    neutral=predictions[0][6]) 
                #print("Predicciones: ", frame_dic)
                #print(emotion_prediction)
                
                # Mostrar en la imagen un texto de la emoción más significativa
                ubicacion = (x, y)
                font = cv2.FONT_HERSHEY_TRIPLEX
                tamañoLetra = 2
                colorLetra = (221, 82, 196)
                grosorLetra = 7
                # Escribir texto con la emoción más significativa
                cv2.putText(imTemp, emotion_prediction, ubicacion, font, tamañoLetra, colorLetra, grosorLetra)
                pt1 = (x, y)
                pt2 = (x + w, y + h)
                color = (23, 200, 54)
                thickness = 10
                cv2.rectangle(imTemp, pt1, pt2, color, thickness)

                # -------Adding captured data----------------

                try:
                    imagesData = dict()
                    imagesData["image_path"] = image_path
                    imagesData["emotions"] = frame_dic
                    imagesData["Location"] = faces_detected

                    listaX = []
                    listaX.append(imagesData)  # Sublista para agregar los elementos a la lista padre
                    photoData.append(listaX)
    
                    
                except:
                    print("Este usuario no existe dentro de la base de datos")
                    return

                # -------Adding captured data----------------

            # Mostrar la imagen con los cuadros en los rostros y el texto de la emoción más significativa
            plt.imshow(imTemp)
            plt.show()

            j += 5
        count += 1
    print("Datos recopilados")
    return photoData



###Funciones importantes

In [104]:
def chooseNameObject(name):
    """
    Retrieves an object from a binary file based on the provided name.

    Opens a binary file containing Interviewed objects and iterates through
    the file to find an object with a matching name. Once a match is found,
    the function returns the object.

    Parameters:
    name (str): The name of the object to retrieve.

    Returns:
    Interviewed or None: The Interviewed object with the matching name, or None if not found.
    """
    userName = ""
    f = open("/content/drive/MyDrive/projectFiles/IIIS-Testing/testingData.bin", "rb")
    f.seek(0)
    flag = 0
    while flag ==0:
        try:
            e = pickle.load(f)
            if e.name == name:
              return e
        except:
            flag = 1
    f.close

def loadIntervieweds():
    """
    Loads Interviewed objects from a binary file and returns them as a list.

    Opens a binary file containing serialized Interviewed objects and iterates
    through the file, deserializing the objects and adding them to a list.
    The list of Interviewed objects is then returned.

    Parameters:
    None

    Returns:
    list: A list of Interviewed objects loaded from the binary file.
    """
    lista = []
    f = open("/content/drive/MyDrive/projectFiles/IIIS-Testing/testingData.bin", "rb")
    f.seek(0)
    flag = 0
    while flag ==0:
        try:
            e = pickle.load(f)
            lista.append(e)
        except:
            flag = 1
    f.close
    return lista  

def updateData(objtectToRefresh):
  """
  Updates the data of an Interviewed object in the loaded list.

  Retrieves the list of Interviewed objects by calling the loadIntervieweds()
  function. Iterates through the list and finds the object with a matching ID.
  Once found, the function updates the object with the provided object and
  returns the updated list.

  Parameters:
  objtectToRefresh (Interviewed): The Interviewed object with updated data.

  Returns:
  list: The updated list of Interviewed objects.
  """
  lista = loadIntervieweds()
  x = 0
  for i in lista:
    print(i)
    if i.id == objtectToRefresh.id:
      lista[x] = objtectToRefresh
    x += 1
  return lista

def saveInFileData(objtectToRefresh):
    """
    Saves the updated Interviewed object in the binary file.

    Retrieves the updated list of Interviewed objects by calling the updateData()
    function. Truncates the existing binary file and writes the serialized objects
    from the updated list to the file.

    Parameters:
    objtectToRefresh (Interviewed): The Interviewed object to be saved in the file.

    Returns:
    None
    """

    lista = updateData(objtectToRefresh)
    # with open('/content/drive/MyDrive/projectFiles/userData.bin', 'wb'):
        # pass  # limpia el archivo
    with open("/content/drive/MyDrive/projectFiles/IIIS-Testing/testingData.bin", "wb") as f:
        f.truncate()  # limpia el archivo
        for i in lista:
            pickle.dump(i, f)

def imprimir(b):
  print("Listo!")

##Senteces Functions

In [51]:
def callFirstSentence(b):
  clear_output()
  lista1 = loadIntervieweds()
  order_userData(lista1)

  buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
  buttonBack.style.button_color = '#1CA9C9'

  display(buttonBack)
  buttonBack.on_click(main)

In [52]:
def callSecondSentence(b):
  clear_output()
  lista2 = loadIntervieweds()
  sorted_data = sort_by_age_insertion(userList2)
  order_userData(sorted_data)

  buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
  buttonBack.style.button_color = '#1CA9C9'

  display(buttonBack)
  buttonBack.on_click(main)

In [53]:
def callThirdSentence(b):
  clear_output()
  userList3 = loadIntervieweds()
  sort_userList_by_children(userList3)

  buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
  buttonBack.style.button_color = '#1CA9C9'

  display(buttonBack)
  buttonBack.on_click(main)

In [54]:
def callFourthSentence(b):
  clear_output()
  userList4 = loadIntervieweds()
  sort_list = sort_by_children_quicksort(userList4)
  order_userData(sort_list)

  buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
  buttonBack.style.button_color = '#1CA9C9'

  display(buttonBack)
  buttonBack.on_click(main)


In [55]:
def callFifthSentence(b):
  clear_output()
  listWithSublists = []

  userList5 = loadIntervieweds()
  for user in userList5:
    userListGeneral = []

    listUserData = []
    listUserEmotions = []

    userName = user.name
    userId = user.id
    userAge = user.calcularEdad()

    listUserData.append(userName)
    listUserData.append(userId)
    listUserData.append(userAge)


    emotions = user.photoData
    for e in emotions:
      userEmotionsSublist = []
      emotionsList = list(e[0]['emotions'])
      for eL in emotionsList:
        userEmotionsSublist.append(e[0]['emotions'][eL])
    
      listUserEmotions.append(userEmotionsSublist)

    userListGeneral.append(listUserData)
    userListGeneral.append(listUserEmotions)

    listWithSublists.append(userListGeneral)
  
    # for e in emotions:      
    # print(f" {e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%")

  for sublist in listWithSublists:
    for data in sublist:
      index = 0
      for idata in data:
        if isinstance(idata, list):
          print("\n")
          print(f"Image {index+1}")
          print(f"Angry: {idata[0]}")
          print(f"Disgust: {idata[1]}")
          print(f"Fear: {idata[2]}")
          print(f"Happiness: {idata[3]}")
          print(f"Sadness: {idata[4]}")
          print(f"Surprise: {idata[5]}")
          print(f"Neutral: {idata[6]}")
          index += 1
        elif idata == data[0]:
         print("Name: ",idata)
        elif idata == data[1]:
         print("ID: ", idata)
        elif idata == data[2]:
         print("Age: ", idata)
    print("\n")
    print("\n")

  buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
  buttonBack.style.button_color = '#1CA9C9'

  display(buttonBack)
  buttonBack.on_click(main)

In [56]:
def callSixthSentence(b):
  clear_output()
  listWithSublists = []

  userList6 = loadIntervieweds()
  for user in userList6:
    userListGeneral = []

    listUserData = []
    listUserEmotions = []

    userName = user.name
    userId = user.id
    userAge = user.calcularEdad()

    listUserData.append(userName)
    listUserData.append(userId)
    listUserData.append(userAge)


    emotions = user.photoData
    for e in emotions:
      userEmotionsSublist = []
      emotionsList = list(e[0]['emotions'])
      for eL in emotionsList:
        userEmotionsSublist.append(e[0]['emotions'][eL])
    
      listUserEmotions.append(userEmotionsSublist)

    userListGeneral.append(listUserData)
    userListGeneral.append(listUserEmotions)

    listWithSublists.append(userListGeneral)

  def quicksort(list1):
    """
    This function takes a list of items and sorts them using the quicksort algorithm.

    Args:
    list1: List of items to be sorted.

    Returns:
    list: Sorted list of items.
    """
    if len(list1) <= 1:
        return list1
    else:
        pivote = list1[0]
        left = []
        right = []
        for i in range(1, len(list1)):
            if list1[i][4] < pivote[4]:
                left.append(list1[i])
            else:
                right.append(list1[i])
        return quicksort(left) + [pivote] + quicksort(right)

  indice = 0
  for x in listWithSublists:
    listaOrdenada = quicksort(listWithSublists[indice][1])
    listWithSublists[indice].pop(1)
    listWithSublists[indice].insert(1, listaOrdenada)
    indice += 1

  for sublist in listWithSublists:
    for data in sublist:
      index = 0
      for idata in data:
        if isinstance(idata, list):
          print(f"Image {index+1}")
          print(f"Angry: {idata[0]}")
          print(f"Disgust: {idata[1]}")
          print(f"Fear: {idata[2]}")
          print(f"Happiness: {idata[3]}")
          print(f"Sadness: {idata[4]}")
          print(f"Surprise: {idata[5]}")
          print(f"Neutral: {idata[6]}")
          index += 1
        elif idata == data[0]:
          print("Name: ",idata)
        elif idata == data[1]:
          print("ID: ", idata)
        elif idata == data[2]:
          print("Age: ", idata)
      print("\n")

  buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
  buttonBack.style.button_color = '#1CA9C9'

  display(buttonBack)
  buttonBack.on_click(main)

In [57]:
def callSeventhSentence(b):
  clear_output()
  execute_graphic_quarter()


  buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
  buttonBack.style.button_color = '#1CA9C9'

  display(buttonBack)
  buttonBack.on_click(main)

In [90]:
def callNinthSentence(b):
  clear_output()
  agregarNuevoUsuario("")

  buttonBack = widgets.Button(description='Home', disabled=False, button_style='', tooltip='Click me', icon='check')
  buttonBack.style.button_color = '#1CA9C9'

  display(buttonBack)
  buttonBack.on_click(main)

In [59]:
def callTenthSentence(b):
  clear_output()
  userList10 = loadIntervieweds()
  hire_candidate(userList10)

##Funcion Imprimir Consultas

In [60]:
def showSentencesButton(b):
  label = widgets.Label(value="User data (First and Second Stage Project): ")
  button1 = widgets.Button(description="Sentence 1", disabled=False, button_style='', tooltip='Click me')
  button2 = widgets.Button(description="Sentence 2", disabled=False, button_style='', tooltip='Click me')
  button3 = widgets.Button(description="Sentence 3", disabled=False, button_style='', tooltip='Click me')
  button4 = widgets.Button(description="Sentence 4", disabled=False, button_style='', tooltip='Click me')
  button5 = widgets.Button(description="Sentence 5", disabled=False, button_style='', tooltip='Click me')
  button6 = widgets.Button(description="Sentence 6", disabled=False, button_style='', tooltip='Click me')
  button7 = widgets.Button(description="Sentence 7", disabled=False, button_style='', tooltip='Click me')
  button9 = widgets.Button(description="Sentence 9", disabled=False, button_style='', tooltip='Click me')
  button10 = widgets.Button(description="Sentence 10", disabled=False, button_style='', tooltip='Click me')
  buttonExit = widgets.Button(description="Home", disabled=False, button_style='', tooltip='Click me')
  buttonExit.style.button_color = "#00A3D4"

  clear_output()
  display(label)
  display(button1)
  display(button2)
  display(button3)
  display(button4)
  display(button5)
  display(button6)
  display(button7)
  display(button9)
  display(button10)
  display(buttonExit)

  button1.on_click(callFirstSentence)
  button2.on_click(callSecondSentence)
  button3.on_click(callThirdSentence)
  button4.on_click(callFourthSentence)
  button5.on_click(callFifthSentence)
  button6.on_click(callSixthSentence)
  button7.on_click(callSeventhSentence)
  button7.on_click(callNinthSentence)
  button10.on_click(callTenthSentence)
  

  buttonExit.on_click(main)
  #listOfUsers = loadIntervieweds()


##Consultas Binary Trees

In [61]:
def callFirstSentenceBT(b):
  clear_output()
  persons = loadIntervieweds()
  display_binary_tree_with_widgets(persons)


  buttonExit = widgets.Button(description="Home", disabled=False, button_style='', tooltip='Click me')
  buttonExit.style.button_color = "#00A3D4"
  display(buttonExit)
  buttonExit.on_click(main)

In [62]:
def callSecondSentenceBT(b):
  clear_output()
  persons = loadIntervieweds()
  build_binary_tree_by_happiness(persons)

  buttonExit = widgets.Button(description="Home", disabled=False, button_style='', tooltip='Click me')
  buttonExit.style.button_color = "#00A3D4"
  display(buttonExit)
  buttonExit.on_click(main)

In [63]:
def callThirdSentenceBT(b):
  clear_output()
  persons = loadIntervieweds()
  display_binary_tree_in_order_with_widgets(persons)

  buttonExit = widgets.Button(description="Home", disabled=False, button_style='', tooltip='Click me')
  buttonExit.style.button_color = "#00A3D4"
  display(buttonExit)
  buttonExit.on_click(main)

In [64]:
def callFourthSentenceBT(b):
  clear_output()
  persons = loadIntervieweds()
  display_binary_tree_preorder_with_widgets(persons)

  buttonExit = widgets.Button(description="Home", disabled=False, button_style='', tooltip='Click me')
  buttonExit.style.button_color = "#00A3D4"
  display(buttonExit)
  buttonExit.on_click(main)

In [65]:
def callFifthSentenceBT(b):
  clear_output()

  def showData(b):
    clear_output()
    persons = loadIntervieweds()
    search_person_by_id_in_tree_with_widgets(persons, input.value)
    
    display(buttonExit)
    buttonExit.on_click(main)

  input = widgets.Text(value="Id", description="ID to look: ", placeholder="Enter ID to look")
  buttonLook = widgets.Button(description="Look user")

  buttonExit = widgets.Button(description="Home", disabled=False, button_style='', tooltip='Click me')
  buttonExit.style.button_color = "#00A3D4"
  display(input)
  display(buttonLook)
  display(buttonExit)

  buttonLook.on_click(showData)
  buttonExit.on_click(main)

In [66]:
def callSixthSentenceBT(b):
  clear_output()

  def showData(b):
    clear_output()
    persons = loadIntervieweds()
    search_person_by_name_in_tree_with_widgets(persons, name.value)
    
    display(buttonExit)
    buttonExit.on_click(main)

  name = widgets.Text(value="Name", description="Name to look: ", placeholder="Enter name to look")
  buttonLook = widgets.Button(description="Look user")

  buttonExit = widgets.Button(description="Home", disabled=False, button_style='', tooltip='Click me')
  buttonExit.style.button_color = "#00A3D4"
  display(name)
  display(buttonLook)
  display(buttonExit)

  buttonLook.on_click(showData)
  buttonExit.on_click(main)

In [67]:
def callSeventhSentenceBT(b):
  clear_output()
  persons = loadIntervieweds()
  display_person_with_highest_happiness(persons)

  buttonExit = widgets.Button(description="Home", disabled=False, button_style='', tooltip='Click me')
  buttonExit.style.button_color = "#00A3D4"
  display(buttonExit)
  buttonExit.on_click(main)

##Binary Tree Menu Show Data

In [68]:
def showSentencesButtonBT(b):
  label = widgets.Label(value="User data by binary trees: ")
  button1 = widgets.Button(description="Sentence 1", disabled=False, button_style='', tooltip='Click me')
  button2 = widgets.Button(description="Sentence 2", disabled=False, button_style='', tooltip='Click me')
  button3 = widgets.Button(description="Sentence 3", disabled=False, button_style='', tooltip='Click me')
  button4 = widgets.Button(description="Sentence 4", disabled=False, button_style='', tooltip='Click me')
  button5 = widgets.Button(description="Sentence 5", disabled=False, button_style='', tooltip='Click me')
  button6 = widgets.Button(description="Sentence 6", disabled=False, button_style='', tooltip='Click me')
  button7 = widgets.Button(description="Sentence 7", disabled=False, button_style='', tooltip='Click me')
  buttonExit = widgets.Button(description="Home", disabled=False, button_style='', tooltip='Click me')
  buttonExit.style.button_color = "#00A3D4"

  clear_output()
  display(label)
  display(button1)
  display(button2)
  display(button3)
  display(button4)
  display(button5)
  display(button6)
  display(button7)
  display(buttonExit)

  button1.on_click(callFirstSentenceBT)
  button2.on_click(callSecondSentenceBT)
  button3.on_click(callThirdSentenceBT)
  button4.on_click(callFourthSentenceBT)
  button5.on_click(callFifthSentenceBT)
  button6.on_click(callSixthSentenceBT)
  button7.on_click(callSeventhSentenceBT)

  buttonExit.on_click(main)
  #listOfUsers = loadIntervieweds()


# Objetos Gráficos



1.   Debe implementar Widgets para la información a recolectar y mostrar para cada uno de los requerimientos del proyecto. A continuación, se listan los principales tipos de widgets que debe utilizar en el proyecto.
  

    *   Numeric widgets
    *   Boolean widgets
    *   Selection widgets
    *   String widgets
    *   Image
    *   Tag widgets
    *   Date picker
    *   Time picker
    *   Color picker
    *   File Upload
    *   Container/Layout widgets



##Consultas (Codigo)

###**1)**

In [106]:
from PIL import Image
import ipywidgets as widgets
from IPython.display import display

userList1 = loadIntervieweds()

def order_userData(userList):
    """
    Displays the captured information using ipywidgets for a more interactive experience.

    Args:
    userList: userList is a general list containing the collected user data.

    Output:
    Show each individual data with its respective title or key using ipywidgets.
    """

    for i in userList:

        name = widgets.Text(value=i.name, description='Name:', disabled=True)
        id = widgets.Text(value=i.id, description='ID:', disabled=True)
        birthDay = widgets.Text(value=i.birthDay, description='BirthDay:', disabled=True)
        age = widgets.Text(value=str(i.calcularEdad()), description='Age (In years old):', disabled=True)
        nacionality = widgets.Text(value=i.nacionality, description='Nacionality:', disabled=True)
        status = widgets.Text(value='Hired' if i.accepted else 'Not hired', description='Status:', disabled=True)

        cellphoneLabel = widgets.Label(value="Cellphone:")
        cellphoneData = []
        for x in i.cellphones:
            number = str(x)
            data = widgets.Text(value=f"{number}: {i.cellphones[x]}", disabled=True)
            cellphoneData.append(data)

        emailLabel = widgets.Label(value="Emails:")
        emailData = []
        for x in i.emails:
            data = widgets.Text(value=x, disabled=True)
            emailData.append(data)

        address = widgets.Text(value=i.address, description='Location:', disabled=True)
        civilStatus = widgets.Text(value=i.civilStatus, description='Civil Status:', disabled=True)

        if i.childrenData == {}:
            childrenData = [widgets.Text(value="Dont have children")]
        else:
            childrenData = []
            childrenList = list(i.childrenData)
            childrenOrder = 0
            while childrenOrder < len(childrenList):
                gender = widgets.Text(value=i.childrenData[childrenList[childrenOrder]]['gender'], description='Gender:', disabled=True)
                birthDayChild = widgets.Text(value=i.childrenData[childrenList[childrenOrder]]['birthDay'], description='BirthDay:', disabled=True)
                childrenOrder += 1
                childrenData.append(widgets.VBox([widgets.Label(value=childrenList[childrenOrder-1]), gender, birthDayChild]))

        imageWidgets = []
        for y in i.photoData:
            for z in y:
                urlImg = list(z.items())[0][1]
                emotionsDict = list(z.items())[1][1]
                locationFace = list(z.items())[2][1]

                img = Image.open(urlImg)
                imgWidget = widgets.Image(value=img._repr_png_(), format='png', width=300, height=300)
                emotions = widgets.VBox([widgets.Label(value="Emotions:")] +
                                        [widgets.Text(value=f"{e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%")
                                         for e in emotionsDict.keys()])
                location = widgets.Text(value=str(locationFace), description='Location face:', disabled=True)
                imageWidgets.append(widgets.VBox([imgWidget, emotions, location]))

        display(widgets.VBox([
            widgets.Label(value="DATA:"),
            name,
            id,
            birthDay,
            age,
            nacionality,
            status,
            cellphoneLabel,
            *cellphoneData,
            emailLabel,
            *emailData,
            address,
            civilStatus,
            *childrenData,
            widgets.Label(value="Images Captured: ------------------------------------------------------"),
            *imageWidgets
        ]))

        print("\n")

###**2)**

In [70]:
userList2 = loadIntervieweds()

def sort_by_age_insertion(lst):
    """
    sort a list of objects by insertion method according to the user's age 

    Args:
    lst = lst is a list of objects to sort

    Return:
    lst already ordered 
    """
    for i in range(0, len(lst)):
        current_value = lst[i]
        current_age = current_value.calcularEdad()
        j = i - 1
        while j >= 0 and lst[j].calcularEdad() > current_age:
            lst[j+1] = lst[j]
            j -= 1
        lst[j+1] = current_value
    return lst

# sorted_data = sort_by_age_insertion(userList2)
# order_userData(sorted_data)

###**3)**

In [71]:
userList3 = loadIntervieweds()

def sort_userList_by_children(userList):
    """
    Sorts the given userList in descending order based on the number of children of each user.
    
    Args:
        userList (list): The list of user data to be sorted.
    
    Returns:
        None: The function modifies the given userList in-place.
    """
    n = len(userList)
    for i in range(n-1):
        for j in range(n-i-1):
            if len(list(userList[j].childrenData)) < len(list(userList[j+1].childrenData)):
                userList[j], userList[j+1] = userList[j+1], userList[j]

    order_userData(userList)


###**4)**

In [72]:
userList4 = loadIntervieweds()

def sort_by_children_quicksort(lst):
    """
    Sort list in descending order of the number of children of each user using the insertion method

    Args:
    lst: lst = lst is a list to sort

    Return:
    List already sorted
    """
    if len(lst) <= 1:
        return lst
    else:
        pivot = len(list(lst[0].childrenData))
        left_lst = []
        right_lst = []
        for i in range(1, len(lst)):
            current_children = len(list(lst[i].childrenData))
            if current_children > pivot:
                left_lst.append(lst[i])
            else:
                right_lst.append(lst[i])
        return sort_by_children_quicksort(left_lst) + [lst[0]] + sort_by_children_quicksort(right_lst)

sorted_data = sort_by_children_quicksort(userList4)

order_userData(sorted_data)

###**5)**

In [73]:
"""
This block obtains for each person interviewed the disordered emotions and stores them in a list (one list per person) with sub-lists,
in the first sub-list it places the identification, name, sex and age of the person interviewed, in the following sub-lists it places
the emotions collected from each person.
"""

listWithSublists = []

userList5 = loadIntervieweds()
for user in userList5:
  userListGeneral = []

  listUserData = []
  listUserEmotions = []

  userName = user.name
  userId = user.id
  userAge = user.calcularEdad()

  listUserData.append(userName)
  listUserData.append(userId)
  listUserData.append(userAge)


  emotions = user.photoData
  for e in emotions:
    userEmotionsSublist = []
    emotionsList = list(e[0]['emotions'])
    for eL in emotionsList:
      userEmotionsSublist.append(e[0]['emotions'][eL])
    
    listUserEmotions.append(userEmotionsSublist)

  userListGeneral.append(listUserData)
  userListGeneral.append(listUserEmotions)

  listWithSublists.append(userListGeneral)
  
  # for e in emotions:      
    # print(f" {e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%")

for sublist in listWithSublists:
  for data in sublist:
    index = 0
    for idata in data:
      if isinstance(idata, list):
          print("\n")
          print(f"Image {index+1}")
          print(f"Angry: {idata[0]}")
          print(f"Disgust: {idata[1]}")
          print(f"Fear: {idata[2]}")
          print(f"Happiness: {idata[3]}")
          print(f"Sadness: {idata[4]}")
          print(f"Surprise: {idata[5]}")
          print(f"Neutral: {idata[6]}")
          index += 1
      elif idata == data[0]:
        print("Name: ",idata)
      elif idata == data[1]:
        print("ID: ", idata)
      elif idata == data[2]:
        print("Age: ", idata)
  print("\n")
  print("\n")


Name:  Nahum
ID:  1
Age:  18


Image 1
Angry: 0.033173225820064545
Disgust: 0.001182072563096881
Fear: 0.01760767586529255
Happiness: 0.5515099167823792
Sadness: 0.054350581020116806
Surprise: 0.006464329082518816
Neutral: 0.3357122242450714


Image 2
Angry: 0.07952951639890671
Disgust: 0.0015889930073171854
Fear: 0.04107232391834259
Happiness: 0.13480599224567413
Sadness: 0.10488099604845047
Surprise: 0.010554105043411255
Neutral: 0.6275680661201477


Image 3
Angry: 0.0874834731221199
Disgust: 0.0008345521637238562
Fear: 0.0504942387342453
Happiness: 0.02764720842242241
Sadness: 0.13712604343891144
Surprise: 0.007028753403574228
Neutral: 0.689385712146759


Image 4
Angry: 0.07915515452623367
Disgust: 0.0010844710050150752
Fear: 0.0369931198656559
Happiness: 0.08201191574335098
Sadness: 0.10625918954610825
Surprise: 0.007181216962635517
Neutral: 0.6873148679733276




Name:  Cristina
ID:  2
Age:  19


Image 1
Angry: 0.0002844849950633943
Disgust: 2.111829644491081e-06
Fear: 0.000635356

###**6)**

In [74]:
def quicksort(list1):
    """
    This function takes a list of items and sorts them using the quicksort algorithm.

    Args:
    list1: List of items to be sorted.

    Returns:
    list: Sorted list of items.
    """
    if len(list1) <= 1:
        return list1
    else:
        pivote = list1[0]
        left = []
        right = []
        for i in range(1, len(list1)):
            if list1[i][4] < pivote[4]:
                left.append(list1[i])
            else:
                right.append(list1[i])
        return quicksort(left) + [pivote] + quicksort(right)

indice = 0
for x in listWithSublists:
  listaOrdenada = quicksort(listWithSublists[indice][1])
  listWithSublists[indice].pop(1)
  listWithSublists[indice].insert(1, listaOrdenada)
  indice += 1

for sublist in listWithSublists:
  for data in sublist:
    index = 0
    for idata in data:
      if isinstance(idata, list):
          print(f"Image {index+1}")
          print(f"Angry: {idata[0]}")
          print(f"Disgust: {idata[1]}")
          print(f"Fear: {idata[2]}")
          print(f"Happiness: {idata[3]}")
          print(f"Sadness: {idata[4]}")
          print(f"Surprise: {idata[5]}")
          print(f"Neutral: {idata[6]}")
          index += 1
      elif idata == data[0]:
        print("Name: ",idata)
      elif idata == data[1]:
        print("ID: ", idata)
      elif idata == data[2]:
        print("Age: ", idata)
  print("\n")

Name:  Nahum
ID:  1
Age:  18
Image 1
Angry: 0.033173225820064545
Disgust: 0.001182072563096881
Fear: 0.01760767586529255
Happiness: 0.5515099167823792
Sadness: 0.054350581020116806
Surprise: 0.006464329082518816
Neutral: 0.3357122242450714
Image 2
Angry: 0.07952951639890671
Disgust: 0.0015889930073171854
Fear: 0.04107232391834259
Happiness: 0.13480599224567413
Sadness: 0.10488099604845047
Surprise: 0.010554105043411255
Neutral: 0.6275680661201477
Image 3
Angry: 0.07915515452623367
Disgust: 0.0010844710050150752
Fear: 0.0369931198656559
Happiness: 0.08201191574335098
Sadness: 0.10625918954610825
Surprise: 0.007181216962635517
Neutral: 0.6873148679733276
Image 4
Angry: 0.0874834731221199
Disgust: 0.0008345521637238562
Fear: 0.0504942387342453
Happiness: 0.02764720842242241
Sadness: 0.13712604343891144
Surprise: 0.007028753403574228
Neutral: 0.689385712146759


Name:  Cristina
ID:  2
Age:  19
Image 1
Angry: 2.3164811864262447e-05
Disgust: 4.9364821563813166e-08
Fear: 2.4555090931244195e-0

###**7)**

In [75]:
import plotly.graph_objects as go

def execute_graphic_quarter():
    def graphic_quarter(numbers, labels, pos, userName):
        """
        This function takes a list of numbers and labels, along with a position and a name, and plots a bar chart 
        using Plotly.

        Args:
        numbers: A list of numbers representing the values of the bars.
        labels: A list of labels representing the names of the bars.
        pos: A list of positions representing the location of the bars in the chart.
        userName: A string representing the name of the chart.

        Output:
        Show user graphic
        """
        fig = go.Figure(data=[go.Bar(x=labels, y=numbers)])
        fig.update_layout(title=f'Average Emotions ({userName}) - 0/10')
        fig.show()

    userList7 = loadIntervieweds()

    for i in userList7:
        n = 1 #Length

        avFear = 0
        avDisgust = 0
        avAngry = 0
        avHappiness = 0
        avSadness = 0
        avSurprise = 0 
        avNeutral = 0

        nombre = i.name
        for j in i.photoData:
            emotions = list(j[0]["emotions"].keys())
            for z in emotions:
                if z == "fear":
                    avFear += j[0]["emotions"][z]
                elif z == "disgust":
                    avDisgust += j[0]["emotions"][z]
                elif z == "angry":
                    avAngry += j[0]["emotions"][z]
                elif z == "happiness":
                    avHappiness += j[0]["emotions"][z]
                elif z == "sadness":
                    avSadness += j[0]["emotions"][z]
                elif z == "surprise":
                    avSurprise += j[0]["emotions"][z]
                elif z == "neutral":
                    avNeutral  += j[0]["emotions"][z]
                n += 1

        avFear = avFear / n
        avDisgust = avDisgust / n
        avAngry = avAngry / n
        avHappiness = avHappiness / n
        avSadness = avSadness / n
        avSurprise = avSurprise / n
        avNeutral = avNeutral / n

        listaEmociones = [avFear*100, avDisgust*100, avAngry*100, avHappiness*100, avSadness*100, avSurprise*100, avNeutral*100]
        labels = ['Fear', 'Disgust', 'Angry', 'Happiness','Sadness', 'Surprise', 'Neutral']
        graphic_quarter(listaEmociones, labels, [], nombre)
        print(" ")

###**9)**

In [103]:
def agregarNuevoUsuario(b):

  outputCellphone = widgets.Output()
  outputEmail = widgets.Output()
  outputChildren = widgets.Output()

  email_inputs = list() 
  cellphone_inputs = dict()
  children_dict = dict()


  name = widgets.Text(
    value='Name here!', 
    placeholder='Type your name',  
    description='Name:',
    disabled=False         
  )

  id = widgets.Text(
    value='ID here!', 
    placeholder='Type your ID',  
    description='ID:',
    disabled=False         
  )

  birthDay =  widgets.Text(
    value='DD-MM-YYYY', 
    placeholder='DD-MM-YYYY',  
    description='BirthDay:',
    disabled=False         
  )

  gender = widgets.Dropdown(
    options = [ 'Male','Female'],
    value='Female',
    description='Gender:',
    disabled=False,
  )
  

  nacionality = widgets.Dropdown(
    options = [ 'Argentina','Belice','Bolivia','Brasil','Canadá','Chile','Colombia','Costa Rica','Cuba','Dominica','Ecuador','El Salvador','Estados Unidos','Guatemala','Haití','Honduras','Jamaica','México','Nicaragua','Panamá','Paraguay','Perú','República Dominicana','Trinidad y Tobago','Uruguay','Venezuela'],
    value='Costa Rica',
    description='Nacionality:',
    disabled=False,
  )
  
  address = widgets.Textarea(
    value='Location here',
    placeholder='Type your location',
    description='Location:',
    disabled=False
   )
  
  nativeLenguage = widgets.Text(
    value='Spanish', 
    placeholder='Type your Native Lenguage',  
    description='Native Lenguage:',
    style={'description_width':'initial'},
    disabled=False         
  )
  civilStatus = widgets.Dropdown(
    options=['Married', 'Divorced', 'Single', 'Other'],
    value='Single',
    description='Civil Status:',
    disabled=False,
    )
  
  state = widgets.ToggleButtons(
    options=['Accepted', 'Denied'],
    description='State: ',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    #     icons=['check'] * 3
    ) 
  
  cellphoneData = widgets.BoundedIntText(
      value=1,              #Inicial value
      min=0,                #Min Value
      max=2,               #Max Value
      step=1,               #Step
      description='Quantity cellphone numbers:',
      style={'description_width':'initial'},
      disabled=False
      )

  emailList = widgets.BoundedIntText(
      value=0,              #Inicial value
      min=0,                #Min Value
      max=2,               #Max Value
      step=1,               #Step
      description='Quantity emails:',
      style={'description_width':'initial'},
      disabled=False
      )
  childrenQuantity = widgets.BoundedIntText(
      value=0,              #Inicial value
      min=0,                #Min Value
      max=14,               #Max Value
      step=1,               #Step
      description='Quantity children:',
      style={'description_width':'initial'},
      disabled=False
      )
    
  buttonCellphones = widgets.Button(
    description='Load cellphone Input',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  buttonEmails = widgets.Button(
    description='Load email Inputs',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  buttonChildren = widgets.Button(
    description='Load Children Inputs',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )

  clear_output()
  display(name, id, gender, birthDay, nacionality, address, nativeLenguage, civilStatus, cellphoneData, buttonCellphones, emailList, buttonEmails, childrenQuantity, buttonChildren)

  button = widgets.Button(
    description='Camera',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  button.style.button_color = '#61FF33'

  buttonH = widgets.Button(
    description='Home',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  buttonH.style.button_color = '#1CA9C9'

  #----Showing quantity inputs for cellphone-----------
  def showCellphoneInputs(b, cellphone_inputs):
    titulo_widget = widgets.Label(value="Cellphone Inputs")
    with outputCellphone:
      clear_output()
      if cellphoneData.value == 0:
        pass
      elif cellphoneData.value == 1:
        display(titulo_widget)

        cellphone = widgets.Text(
          value=f'Cellphone number 1', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone_inputs["Cellphone 1"] = cellphone
        display(cellphone)

      elif cellphoneData.value == 2:
        display(titulo_widget)

        cellphone = widgets.Text(
          value=f'Cellphone number 1', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone2 = widgets.Text(
          value=f'Cellphone 2', 
          placeholder='Type your cellphone',  
          description=f'Cellphone 2:',
          style={'description_width':'initial'},
          disabled=False         
          )
        cellphone_inputs["Cellphone 1"] = cellphone
        cellphone_inputs["Cellphone 2"] = cellphone2
        display(widgets.VBox([cellphone, cellphone2]))  
          


  buttonCellphones.on_click(lambda b: showCellphoneInputs(b, cellphone_inputs))
  #----------------------------------------------------
  display(outputCellphone)

  #--Get cellphone data-------------------
  def obtenerValoresCellphones():
    valores_cellphones = {key: cellphone.value for key, cellphone in cellphone_inputs.items()}
    return valores_cellphones
  #-----------------------------------


  #----Showing quantity inputs for cellphone-----------
  def showEmailsInputs(b, email_inputs):
    titulo_widget = widgets.Label(value="Emails Inputs")
    with outputEmail:
      clear_output()
      if emailList.value == 0:
        pass
      elif emailList.value == 1:
        display(titulo_widget)

        email = widgets.Text(
          value=f'Email 1', 
          placeholder='Type your email',  
          description=f'Email 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        email_inputs.append(email)
        display(email)



      elif emailList.value == 2:
        display(titulo_widget)

        email = widgets.Text(
          value=f'Email 1', 
          placeholder='Type your email',  
          description=f'Email 1:',
          style={'description_width':'initial'},
          disabled=False         
          )
        
        email2 = widgets.Text(
          value=f'Email 2', 
          placeholder='Type your email',  
          description=f'Email 2:',
          style={'description_width':'initial'},
          disabled=False         
          )
        email_inputs.extend([email, email2])
        display(widgets.VBox([email, email2]))     

  buttonEmails.on_click(lambda b: showEmailsInputs(b, email_inputs))
  #----------------------------------------------------
  display(outputEmail)


  #--Get email data-------------------
  def obtenerValoresEmails():
    valores_emails = [email.value for email in email_inputs]
    return valores_emails
  #-----------------------------------


  #----Show Quantity Children inputs----------------
  def showChildrenInputs(b, children_dict):
    with outputChildren:
        clear_output()
        titulo_widget = widgets.Label(value="Children Inputs")
        display(titulo_widget)

        quantity = childrenQuantity.value

        child_widgets = []

        for i in range(quantity):
            name = widgets.Text(
                value='Name here!', 
                placeholder='Type your name',  
                description='Name:',
                disabled=False         
            )

            gender = widgets.Dropdown(
                options=['Male', 'Female'],
                value='Female',
                description='Gender:',
                disabled=False,
            )

            birthDay = widgets.Text(
                value='DD-MM-YYYY',
                placeholder='DD-MM-YYYY',
                description='BirthDay:',
                disabled=False
            )

            children_dict[str(i+1)] = {
                'name': name,
                'gender': gender,
                'birthDay': birthDay
            }

            if i > 0:
                titulo_widget_child = widgets.Label(value=f"Child {i+1}:")
                child_widgets.append(titulo_widget_child)

            child_widgets.extend([name, gender, birthDay])

        display(widgets.VBox(child_widgets))
    

  def getChildrenData():
    data_dict = {}
    
    for key, value in children_dict.items():
        child_data = {}
        child_data['name'] = value['name'].value
        child_data['gender'] = value['gender'].value
        child_data['birthDay'] = value['birthDay'].value
        data_dict[key] = child_data
    
    return data_dict

  buttonChildren.on_click(lambda b: showChildrenInputs(b, children_dict))
#-------------------------------------------------
  display(outputChildren)


  #----Save Data----------------------
  def guardar(b):
    newInterviewed = Interviewed(name.value, id.value, gender.value, birthDay.value, nacionality.value, address.value, nativeLenguage.value, state.value, civilStatus.value, {}, None)
    newInterviewed.emails = obtenerValoresEmails()
    newInterviewed.cellphones = obtenerValoresCellphones()
    newInterviewed.childrenData = getChildrenData()
    photoData = process_video_stream(name.value)
    newInterviewed.photoData = photoData 
    userList = loadIntervieweds()
    userFound = False

    for i in range(len(userList)):
        if userList[i].id == newInterviewed.id:
            userList[i] = newInterviewed
            userFound = True
            saveInFileData(newInterviewed)
            break

    if not userFound:
        with open("/content/drive/MyDrive/projectFiles/personsCapturedData/Archivo_Entrevistado.bin", "ab") as f:
            pickle.dump(newInterviewed, f)
            display(widgets.Label(value="Nuevo usuario agregado correctamente"))

  def ejecutarFunciones(b):
    guardar(b)
    main(b)
  
  def home(b):
    main(b)

  button.on_click(ejecutarFunciones)
  buttonH.on_click(home)

  button.on_click(ejecutarFunciones)
  #------------------------------------
  display(widgets.HBox([button, buttonH]))

###**10)**

In [77]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

def hire_candidate(list):
    accepted_user_list = []
    index = 0  # Variable para llevar el seguimiento del índice del entrevistado actual

    def display_candidate_data(candidate):
        nonlocal index
        clear_output()  # Borrar la información anterior de la entrevista
        output = widgets.Output()

        with output:
            display(widgets.Label("DATA:"))
            display(widgets.Label(f"Name: {candidate.name}"))  # Imprimir nombre de usuario que posee la lista
            display(widgets.Label(f"ID: {candidate.id}"))
            display(widgets.Label(f"BirthDay entered: {candidate.birthDay}"))
            display(widgets.Label(f"Age (In years old): {candidate.calcularEdad()}"))
            display(widgets.Label(f"Nacionality: {candidate.nacionality}"))

            if candidate.accepted:
                display(widgets.Label("Status: Hired"))
            else:
                display(widgets.Label("Status: Not hired"))

            display(widgets.Label("Cellphone:"))
            for x in candidate.cellphones:
                number = str(x)
                display(widgets.Label(f" {x}: {number} "))

            display(widgets.Label("Emails:"))
            for x in candidate.emails:
                display(widgets.Label(f" {x}"))

            display(widgets.Label(f"Location: {candidate.address}"))
            display(widgets.Label(f"Civil Status: {candidate.civilStatus}"))

            if candidate.childrenData == {}:
                display(widgets.Label("Don't have children"))
            else:
                display(widgets.Label("Children:"))
                childrenList = candidate.childrenData
                childrenOrder = 0
                for w in childrenList:
                    display(widgets.Label(f"{w} data:"))
                    display(widgets.Label(f"Gender: {candidate.childrenData[w]['gender']}"))
                    display(widgets.Label(f"BirthDay: {candidate.childrenData[w]['birthDay']}"))
                    childrenOrder += 1

            display(widgets.Label("Images Captured:"))
            for y in candidate.photoData:
                for z in y:
                    urlImg = z['image_path']
                    emotionsDict = z['emotions']
                    locationFace = z['Location']

                    img = Image.open(urlImg)
                    display(img)

                    display(widgets.Label("Emotions:"))
                    for e in emotionsDict.keys():
                        display(widgets.Label(f" {e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%"))

                    display(widgets.Label("Location face:"))
                    display(widgets.Label(str(locationFace)))

            display(widgets.Label("\n"))
            display(widgets.Label("Will the person be hired?"))

            state = widgets.ToggleButtons(
                options=['Accepted', 'Denied'],
                description='State: ',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
            )
            display(state)

            save_button = widgets.Button(description='Save')

            def handle_save_button_click(b):
                nonlocal index
                if state.value == 'Accepted':
                    candidate.accepted = True
                elif state.value == 'Denied':
                    candidate.accepted = False
                else:
                    print("Invalid option")

                accepted_user_list.append(candidate)
                print("Instance saved successfully.")

                # Salto a la siguiente entrevista
                index += 1
                if index < len(list):
                    display_candidate_data(list[index])
                else:
                    clear_output()
                    print("End of candidate list.")
                    display(buttonH)

            save_button.on_click(handle_save_button_click)
            display(save_button)

        display(output)

    def handle_button_click(button):
        nonlocal index

        if index < len(list):
            display_candidate_data(list[index])
        else:
            clear_output()
            print("End of candidate list.")
            display(buttonH)

    def handle_home_button_click(button):
        clear_output()
        main("")

    # Crear los botones usando ipywidgets
    buttonH = widgets.Button(description='Home')  
    buttonH.style.button_color = "#00A3D4"
    button = widgets.Button(description='Evaluate')
    buttonH.on_click(handle_home_button_click)
    button.on_click(handle_button_click)

    # Mostrar los botones
    display(button)
    display(buttonH)

# Llamar a la función hire_candidate() para ejecutar el código
hire_candidate(loadIntervieweds())

Button(description='Evaluate', style=ButtonStyle())

Button(description='Home', style=ButtonStyle(button_color='#00A3D4'))

# Consultas


1.   Mostrar la información de todas las personas entrevistas (la información debe ser mostrada siendo saca del archivo correspondiente, debe mostrar la información en formato legible, no en estructuras y objetos). Además, debe mostrar toda la información recolectada de la persona, se debe mostrar todas las imágenes que se recolectaron durante las entrevistas, las emociones reconocidas para cada imagen y las coordenadas donde se encuentra los rostros.

2.   Mostrar la información de todas las personas entrevistas ordenada ascendentemente por edad. El método de ordenamiento a utilizar es el método por inserción (la información debe ser mostrada siendo saca de los archivos, no se debe hacer simplemente la impresión en pantalla de la información obtenida de los archivos). Además, de mostrar toda la información recolectada de la persona, se debe mostrar todas las imágenes que se recolectaron durante las entrevistas, las emociones reconocidas para cada imagen y las coordenadas donde se encuentra los rostros.

3.   Mostrar la información de todas las personas entrevistas ordenada descendente mente por la cantidad de hijos. El método de ordenamiento a utilizar es el método por de la bubuja (la información debe ser mostrada siendo saca de los archivos, no se debe hacer simplemente la impresión en pantalla de la información obtenida de los archivos). Además, de mostrar toda la información recolectada de la persona, se debe mostrar todas las imágenes que se recolectaron durante las entrevistas, las emociones reconocidas para cada imagen y las coordenadas donde se encuentra los rostros.

4. Mostrar la información de todas las personas entrevistas ordenada descendentemente por la cantidad de hijos. El método de ordenamiento a utilizar es el método por de la quicksort (la información debe ser mostrada siendo sacada de los archivos, no se debe hacer simplemente la impresión en pantalla de la información obtenida de los archivos). Además, de mostrar toda la información recolectada de la persona, se debe mostrar todas las imágenes que se recolectaron durante las entrevistas, las emociones reconocidas para cada imagen y las coordenadas donde se encuentra los rostros. También, imprimir una lista con sublistas donde la primera sublista tiene la información de las personas que tienen únicamente hijos de género femenino, la segunda sublista tiene información de personas que tienen únicamente hijos de género masculino y la última sublista tiene información de personas que tienen hijos tanto de género femenino como másculino. 

5. Obtener para cada persona entrevistada las emociones sin ordenar y guardar en una lista (una lista por entrevista) con sublistas la siguiente información: en la primera sublista colocar la identificación, el nombre, el género y la edad de la persona entrevistada, en las siguientes sublistas colocar las emociones expresadas por la persona. Por ejemplo, si la persona apareció en 10 fotogramas entonces se deben crear 10 sublistas, cada sublista pertenece a las emociones emitidas por el candidato en un fotograma. 

6. Una vez creada esta listas (una por entrevista) con sublistas del punto anterior, se debe ordenar las sublistas de forma ascendente por la emoción felicidad (happiness) utilizando el técnica de ordenamiento Quicksort.

7. Para cada persona entrevistada muestre un gráfico para comparar las emociones reportadas durante la entrevista.

8. Incluya en el documento escrito la entrevista a dos personas. De aportar toda la información recolectada, las imágenes recolectadas en la entrevista con un cuadro en los rostros presentes. Además, debe mostrar la información de las emociones reportadas. También debe mostrar la información del entrevistador y el evaluador.

9. Hacer una nueva entrevista a una persona. Una persona existente (previamente almacena en los archivos) puede volver a ser entrevistada. Debe guardar la nueva entrevista en el mismo archivo agregando la información de esta nueva entrevista a la lista de entrevistas de dicha persona. Tome en cuenta que el entrevistador puede ser uno que no le ha realizado ninguna entrevista.

10. Agregar una evaluación para indicar si la persona será contratada. Una vez que una persona ha sido entrevistada un evaluador puede revisar toda la información y decir si la persona será contratada (Debe mostrar la información de la persona para que el evaluador la revise y debe aportar una propiedad para la persona que se indica llamada "estado" (el estado puede ser entrevistado, contratado y no contratado). Tome en cuenta que el evaluador puede cambiar el estado de la persona para indicar que está contratado o no contratado. 




# Árboles binarios


1.   Implementar un árbol binario para almacenar de forma ordenada las personas por edad (Cada nodo del árbol debe contener una instancia de la clase Entrevistado).
2.   Implementar un árbol binario para almacenar de forma ordenada las personas por por la emoción de felicidad (Cada nodo del árbol debe contener una instancia de la clase Entrevistado).

3. Implementar una función que muestre la información del árbol implementado en el punto 1 en orden (Debe mostrar la información usando widgets).


4. Implementar una función que muestre la información del árbol implementado en el punto 2 en preorden (Debe mostrar la información usando widgets).


5. Implementar una función que buscar una persona por la identificación en el árbol implementado en el punto 1. 

6. Implementar una función que buscar una persona por el nombre en el árbol implementado en el punto 2.

7. Implementar una función que buscar muestre todas las personas que tienen el grado de felicidad más alto en el árbol implementado en el punto 1. 

###**1)**

In [78]:
def display_binary_tree_with_widgets(persons):
    def build_binary_tree_by_age(persons):
        class BinaryTreeNode:
            def __init__(self, person):
                self.person = person
                self.left = None
                self.right = None

        def insert_node(root, person):
            if root is None:
                return BinaryTreeNode(person)

            if person.calcularEdad() < root.person.calcularEdad():
                root.left = insert_node(root.left, person)
            else:
                root.right = insert_node(root.right, person)

            return root

        root = None
        for person in persons:
            if root is None:
                root = BinaryTreeNode(person)
            else:
                root = insert_node(root, person)
        return root

    def print_binary_tree_in_order(root):
        output = []

        def traverse(node):
            nonlocal output
            if node is not None:
                traverse(node.left)
                output.append(node.person)
                traverse(node.right)

        traverse(root)
        return output

    def display_person_widgets(person):
        name = widgets.Text(value=person.name, description='Name:', disabled=True)
        id = widgets.Text(value=person.id, description='ID:', disabled=True)
        birthDay = widgets.Text(value=person.birthDay, description='BirthDay:', disabled=True)
        age = widgets.Text(value=str(person.calcularEdad()), description='Age (In years old):', disabled=True)
        nacionality = widgets.Text(value=person.nacionality, description='Nacionality:', disabled=True)
        status = widgets.Text(value='Hired' if person.accepted else 'Not hired', description='Status:', disabled=True)

        cellphoneLabel = widgets.Label(value="Cellphone:")
        cellphoneData = []
        for x in person.cellphones:
            number = str(x)
            data = widgets.Text(value=f"{number}: {person.cellphones[x]}", disabled=True)
            cellphoneData.append(data)


        emailLabel = widgets.Label(value="Emails:")
        emailData = []
        for email in person.emails:
            emailData.append(widgets.Text(value=email, disabled=True))

        address = widgets.Text(value=person.address, description='Location:', disabled=True)
        civilStatus = widgets.Text(value=person.civilStatus, description='Civil Status:', disabled=True)

        if person.childrenData == {}:
            childrenData = [widgets.Text(value="Dont have children")]
        else:
            childrenData = []
            for child_name, child_data in person.childrenData.items():
                gender = widgets.Text(value=child_data['gender'], description='Gender:', disabled=True)
                birthDayChild = widgets.Text(value=child_data['birthDay'], description='BirthDay:', disabled=True)
                childrenData.append(widgets.VBox([widgets.Label(value=child_name), gender, birthDayChild]))

        imageWidgets = []
        for image_data in person.photoData:
            urlImg = image_data[0]["image_path"]
            emotionsDict = image_data[0]["emotions"]
            locationFace = image_data[0]["Location"]

            img = Image.open(urlImg)
            imgWidget = widgets.Image(value=img._repr_png_(), format='png', width=300, height=300)
            emotions = widgets.VBox([widgets.Label(value="Emotions:")] +
                                    [widgets.Text(value=f"{e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%")
                                     for e in emotionsDict.keys()])
            location = widgets.Text(value=str(locationFace), description='Location face:', disabled=True)
            imageWidgets.append(widgets.VBox([imgWidget, emotions, location]))

        display(widgets.VBox([
            widgets.Label(value="DATA:"),
            name,
            id,
            birthDay,
            age,
            nacionality,
            status,
            cellphoneLabel,
            *cellphoneData,
            emailLabel,
            *emailData,
            address,
            civilStatus,
            *childrenData,
            widgets.Label(value="Images Captured: ------------------------------------------------------"),
            *imageWidgets
        ]))

        print("\n")

    tree_root = build_binary_tree_by_age(persons)
    tree_data = print_binary_tree_in_order(tree_root)

    # Mostrar de menor a mayor
    tree_widget_ascending = widgets.Output()
    with tree_widget_ascending:
        for person in tree_data:
            display_person_widgets(person)
    display(tree_widget_ascending)

###**2)**

In [79]:
def build_binary_tree_by_happiness(persons):
    class BinaryTreeNode:
        def __init__(self, person):
            self.person = person
            self.left = None
            self.right = None

    def calculate_happiness_average(emotionsList):
        # Calcula el promedio de la emoción de felicidad
        total = 0
        for emotions in emotionsList:
          index = 0
          for x in emotions:
            happiness = x["emotions"]["happiness"]
            index += 1
            total += happiness
        promedio = (total / len(emotionsList)) * 100
        return promedio

        

    def insert_node(root, person):
        if root is None:
            return BinaryTreeNode(person)

        root_happiness = calculate_happiness_average(root.person.photoData)
        person_happiness = calculate_happiness_average(person.photoData)

        if person_happiness < root_happiness:
            root.left = insert_node(root.left, person)
        else:
            root.right = insert_node(root.right, person)

        return root

    def print_ordered_data(node):
        if node is not None:
            print_ordered_data(node.left)
            display_person_data(node.person)
            print_ordered_data(node.right)

    def display_person_data(person):
        name = widgets.Text(value=str(person.name), description='Name:', disabled=True)
        id = widgets.Text(value=str(person.id), description='ID:', disabled=True)
        age = widgets.Text(value=str(person.calcularEdad()), description='Age:', disabled=True)
        # Rest of the code for creating widgets for the person's data

        # Display the widgets
        userData = widgets.HBox([name, id, age]) # Add other widgets as needed

        imageWidgets = []
        for image_data in person.photoData:
            urlImg = image_data[0]["image_path"]
            emotionsDict = image_data[0]["emotions"]
            locationFace = image_data[0]["Location"]

            img = Image.open(urlImg)
            emotions = widgets.VBox([widgets.Label(value="Emotions:")] +
                                    [widgets.Text(value=f"{e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%")
                                     for e in emotionsDict.keys()])
            imageWidgets.append(widgets.VBox([emotions]))

        display(widgets.VBox([
            widgets.Label(value="DATA:"),
            userData,
            widgets.Label(value="Images Captured: ------------------------------------------------------"),
            *imageWidgets,
            widgets.Label(value=" ")
        ]))


    root = None
    for person in persons:
        if root is None:
            root = BinaryTreeNode(person)
        else:
            root = insert_node(root, person)

    print_ordered_data(root)

###**3)**

In [80]:
def display_binary_tree_in_order_with_widgets(persons):
    class BinaryTreeNode:
        def __init__(self, person):
            self.person = person
            self.left = None
            self.right = None

    def build_binary_tree_by_age(persons):
        def insert_node(root, person):
            if root is None:
                return BinaryTreeNode(person)

            if person.calcularEdad() < root.person.calcularEdad():
                root.left = insert_node(root.left, person)
            else:
                root.right = insert_node(root.right, person)

            return root

        root = None
        for person in persons:
            if root is None:
                root = BinaryTreeNode(person)
            else:
                root = insert_node(root, person)
        return root

    def print_binary_tree_in_order(root):
        output = []

        def traverse(node):
            nonlocal output
            if node is not None:
                traverse(node.left)
                output.append(node.person)
                traverse(node.right)

        traverse(root)
        return output
    def calculate_happiness_average(emotionsList):
        # Calcula el promedio de la emoción de felicidad
        for emotions in emotionsList:
          index = 0
          total = 0
          for x in emotions:
            happiness = x["emotions"]["happiness"]
            index += 1
            total += happiness
        promedio = total / len(emotionsList)
        return promedio
    def display_person_widgets(person):
        name = widgets.Text(value=person.name, description='Name:', disabled=True)
        id = widgets.Text(value=person.id, description='ID:', disabled=True)
        birthDay = widgets.Text(value=person.birthDay, description='BirthDay:', disabled=True)
        age = widgets.Text(value=str(person.calcularEdad()), description='Age (In years old):', disabled=True)
        nacionality = widgets.Text(value=person.nacionality, description='Nacionality:', disabled=True)
        status = widgets.Text(value='Hired' if person.accepted else 'Not hired', description='Status:', disabled=True)

        cellphoneLabel = widgets.Label(value="Cellphone:")
        cellphoneData = []
        for x in person.cellphones:
            number = str(x)
            data = widgets.Text(value=f"{number}: {person.cellphones[x]}", disabled=True)
            cellphoneData.append(data)


        emailLabel = widgets.Label(value="Emails:")
        emailData = []
        for email in person.emails:
            emailData.append(widgets.Text(value=email, disabled=True))

        address = widgets.Text(value=person.address, description='Location:', disabled=True)
        civilStatus = widgets.Text(value=person.civilStatus, description='Civil Status:', disabled=True)

        if person.childrenData == {}:
            childrenData = [widgets.Text(value="Dont have children")]
        else:
            childrenData = []
            for child_name, child_data in person.childrenData.items():
                gender = widgets.Text(value=child_data['gender'], description='Gender:', disabled=True)
                birthDayChild = widgets.Text(value=child_data['birthDay'], description='BirthDay:', disabled=True)
                childrenData.append(widgets.VBox([widgets.Label(value=child_name), gender, birthDayChild]))

        imageWidgets = []
        for image_data in person.photoData:
            urlImg = image_data[0]["image_path"]
            emotionsDict = image_data[0]["emotions"]
            locationFace = image_data[0]["Location"]

            img = Image.open(urlImg)
            imgWidget = widgets.Image(value=img._repr_png_(), format='png', width=300, height=300)
            emotions = widgets.VBox([widgets.Label(value="Emotions:")] +
                                    [widgets.Text(value=f"{e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%")
                                     for e in emotionsDict.keys()])
            location = widgets.Text(value=str(locationFace), description='Location face:', disabled=True)
            imageWidgets.append(widgets.VBox([imgWidget, emotions, location]))

        display(widgets.VBox([
            widgets.Label(value="DATA:"),
            name,
            id,
            birthDay,
            age,
            nacionality,
            status,
            cellphoneLabel,
            *cellphoneData,
            emailLabel,
            *emailData,
            address,
            civilStatus,
            *childrenData,
            widgets.Label(value="Images Captured: ------------------------------------------------------"),
            *imageWidgets
        ]))

        print("\n")

    tree_root = build_binary_tree_by_age(persons)
    tree_data = print_binary_tree_in_order(tree_root)

    tree_widget_ascending = widgets.Output()
    with tree_widget_ascending:
        for person in tree_data:
            display_person_widgets(person)

    display(tree_widget_ascending)

###**4)**

In [81]:
def display_binary_tree_preorder_with_widgets(persons):
    class BinaryTreeNode:
        def __init__(self, person):
            self.person = person
            self.left = None
            self.right = None

    def calculate_happiness_average(emotionsList):
        # Calcula el promedio de la emoción de felicidad
        total = 0
        for emotions in emotionsList:
            index = 0
            for x in emotions:
                happiness = x["emotions"]["happiness"]
                index += 1
                total += happiness
        promedio = (total / len(emotionsList)) * 100
        return promedio

    def insert_node(root, person):
        if root is None:
            return BinaryTreeNode(person)

        root_happiness = calculate_happiness_average(root.person.photoData)
        person_happiness = calculate_happiness_average(person.photoData)

        if person_happiness < root_happiness:
            root.left = insert_node(root.left, person)
        else:
            root.right = insert_node(root.right, person)

        return root

    def print_preordered_data(node):
        if node is not None:
            display_person_data(node.person)
            print_preordered_data(node.left)
            print_preordered_data(node.right)

    def display_person_data(person):
        name = widgets.Text(value=person.name, description='Name:', disabled=True)
        id = widgets.Text(value=person.id, description='ID:', disabled=True)
        birthDay = widgets.Text(value=person.birthDay, description='BirthDay:', disabled=True)
        age = widgets.Text(value=str(person.calcularEdad()), description='Age (In years old):', disabled=True)
        nacionality = widgets.Text(value=person.nacionality, description='Nacionality:', disabled=True)
        status = widgets.Text(value='Hired' if person.accepted else 'Not hired', description='Status:', disabled=True)

        cellphoneLabel = widgets.Label(value="Cellphone:")
        cellphoneData = []
        for x in person.cellphones:
            number = str(x)
            data = widgets.Text(value=f"{number}: {person.cellphones[x]}", disabled=True)
            cellphoneData.append(data)


        emailLabel = widgets.Label(value="Emails:")
        emailData = []
        for email in person.emails:
            emailData.append(widgets.Text(value=email, disabled=True))

        address = widgets.Text(value=person.address, description='Location:', disabled=True)
        civilStatus = widgets.Text(value=person.civilStatus, description='Civil Status:', disabled=True)

        if person.childrenData == {}:
            childrenData = [widgets.Text(value="Dont have children")]
        else:
            childrenData = []
            for child_name, child_data in person.childrenData.items():
                gender = widgets.Text(value=child_data['gender'], description='Gender:', disabled=True)
                birthDayChild = widgets.Text(value=child_data['birthDay'], description='BirthDay:', disabled=True)
                childrenData.append(widgets.VBox([widgets.Label(value=child_name), gender, birthDayChild]))

        imageWidgets = []
        for image_data in person.photoData:
            urlImg = image_data[0]["image_path"]
            emotionsDict = image_data[0]["emotions"]
            locationFace = image_data[0]["Location"]

            img = Image.open(urlImg)
            imgWidget = widgets.Image(value=img._repr_png_(), format='png', width=300, height=300)
            emotions = widgets.VBox([widgets.Label(value="Emotions:")] +
                                    [widgets.Text(value=f"{e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%")
                                     for e in emotionsDict.keys()])
            location = widgets.Text(value=str(locationFace), description='Location face:', disabled=True)
            imageWidgets.append(widgets.VBox([imgWidget, emotions, location]))

        display(widgets.VBox([
            widgets.Label(value="DATA:"),
            name,
            id,
            birthDay,
            age,
            nacionality,
            status,
            cellphoneLabel,
            *cellphoneData,
            emailLabel,
            *emailData,
            address,
            civilStatus,
            *childrenData,
            widgets.Label(value="Images Captured: ------------------------------------------------------"),
            *imageWidgets
        ]))

        print("\n")


    root = None
    for person in persons:
        if root is None:
            root = BinaryTreeNode(person)
        else:
            root = insert_node(root, person)

    print_preordered_data(root)

###**5)**

In [82]:
def search_person_by_id_in_tree_with_widgets(persons, id):
    def build_binary_tree_by_age(persons):
        class BinaryTreeNode:
            def __init__(self, person):
                self.person = person
                self.left = None
                self.right = None

        def insert_node(root, person):
            if root is None:
                return BinaryTreeNode(person)

            if person.calcularEdad() < root.person.calcularEdad():
                root.left = insert_node(root.left, person)
            else:
                root.right = insert_node(root.right, person)

            return root

        root = None
        for person in persons:
            if root is None:
                root = BinaryTreeNode(person)
            else:
                root = insert_node(root, person)
        return root

    # def insert_node(root, person):
    #     if root is None:
    #         return BinaryTreeNode(person)

    #     if person.calcularEdad() < root.person.calcularEdad():
    #         root.left = insert_node(root.left, person)
    #     else:
    #         root.right = insert_node(root.right, person)

    #     return root

    def print_binary_tree_in_order(root):
        output = []

        def traverse(node):
            nonlocal output
            if node is not None:
                traverse(node.left)
                output.append(node.person)
                traverse(node.right)

        traverse(root)
        return output

    def display_person_widgets(person):
        name = widgets.Text(value=person.name, description='Name:', disabled=True)
        id = widgets.Text(value=person.id, description='ID:', disabled=True)
        birthDay = widgets.Text(value=person.birthDay, description='BirthDay:', disabled=True)
        age = widgets.Text(value=str(person.calcularEdad()), description='Age (In years old):', disabled=True)
        nacionality = widgets.Text(value=person.nacionality, description='Nacionality:', disabled=True)
        status = widgets.Text(value='Hired' if person.accepted else 'Not hired', description='Status:', disabled=True)

        cellphoneLabel = widgets.Label(value="Cellphone:")
        cellphoneData = []
        for x in person.cellphones:
            number = str(x)
            data = widgets.Text(value=f"{number}: {person.cellphones[x]}", disabled=True)
            cellphoneData.append(data)


        emailLabel = widgets.Label(value="Emails:")
        emailData = []
        for email in person.emails:
            emailData.append(widgets.Text(value=email, disabled=True))

        address = widgets.Text(value=person.address, description='Location:', disabled=True)
        civilStatus = widgets.Text(value=person.civilStatus, description='Civil Status:', disabled=True)

        if person.childrenData == {}:
            childrenData = [widgets.Text(value="Dont have children")]
        else:
            childrenData = []
            for child_name, child_data in person.childrenData.items():
                gender = widgets.Text(value=child_data['gender'], description='Gender:', disabled=True)
                birthDayChild = widgets.Text(value=child_data['birthDay'], description='BirthDay:', disabled=True)
                childrenData.append(widgets.VBox([widgets.Label(value=child_name), gender, birthDayChild]))

        imageWidgets = []
        for image_data in person.photoData:
            urlImg = image_data[0]["image_path"]
            emotionsDict = image_data[0]["emotions"]
            locationFace = image_data[0]["Location"]

            img = Image.open(urlImg)
            imgWidget = widgets.Image(value=img._repr_png_(), format='png', width=300, height=300)
            emotions = widgets.VBox([widgets.Label(value="Emotions:")] +
                                    [widgets.Text(value=f"{e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%")
                                     for e in emotionsDict.keys()])
            location = widgets.Text(value=str(locationFace), description='Location face:', disabled=True)
            imageWidgets.append(widgets.VBox([imgWidget, emotions, location]))

        display(widgets.VBox([
            widgets.Label(value="DATA:"),
            name,
            id,
            birthDay,
            age,
            nacionality,
            status,
            cellphoneLabel,
            *cellphoneData,
            emailLabel,
            *emailData,
            address,
            civilStatus,
            *childrenData,
            widgets.Label(value="Images Captured: ------------------------------------------------------"),
            *imageWidgets
        ]))

        print("\n")

    tree_root = build_binary_tree_by_age(persons)
    tree_data = print_binary_tree_in_order(tree_root)

    # Buscar la persona por identificación
    found_person = None
    for person in tree_data:
        if person.id == id:
            found_person = person
            break

    if found_person:
        display_person_widgets(found_person)
    else:
        print("Persona no encontrada en el árbol.")

###**6)**

In [83]:
def search_person_by_name_in_tree_with_widgets(persons, name):
    class BinaryTreeNode:
        def __init__(self, person):
            self.person = person
            self.left = None
            self.right = None

    def calculate_happiness_average(emotionsList):
        # Calcula el promedio de la emoción de felicidad
        for emotions in emotionsList:
            index = 0
            total = 0
            for x in emotions:
                happiness = x["emotions"]["happiness"]
                index += 1
                total += happiness
        promedio = (total / len(emotionsList)) * 100
        return promedio

    def insert_node(root, person):
        if root is None:
            return BinaryTreeNode(person)

        root_happiness = calculate_happiness_average(root.person.photoData)
        person_happiness = calculate_happiness_average(person.photoData)

        if person_happiness < root_happiness:
            root.left = insert_node(root.left, person)
        else:
            root.right = insert_node(root.right, person)

        return root

    def print_ordered_data(node):
        if node is not None:
            print_ordered_data(node.left)
            display_person_data(node.person)
            print_ordered_data(node.right)

    def display_person_data(person):
        name = widgets.Text(value=person.name, description='Name:', disabled=True)
        id = widgets.Text(value=person.id, description='ID:', disabled=True)
        birthDay = widgets.Text(value=person.birthDay, description='BirthDay:', disabled=True)
        age = widgets.Text(value=str(person.calcularEdad()), description='Age (In years old):', disabled=True)
        nacionality = widgets.Text(value=person.nacionality, description='Nacionality:', disabled=True)
        status = widgets.Text(value='Hired' if person.accepted else 'Not hired', description='Status:', disabled=True)

        cellphoneLabel = widgets.Label(value="Cellphone:")
        cellphoneData = []
        for x in person.cellphones:
            number = str(x)
            data = widgets.Text(value=f"{number}: {person.cellphones[x]}", disabled=True)
            cellphoneData.append(data)


        emailLabel = widgets.Label(value="Emails:")
        emailData = []
        for email in person.emails:
            emailData.append(widgets.Text(value=email, disabled=True))

        address = widgets.Text(value=person.address, description='Location:', disabled=True)
        civilStatus = widgets.Text(value=person.civilStatus, description='Civil Status:', disabled=True)

        if person.childrenData == {}:
            childrenData = [widgets.Text(value="Dont have children")]
        else:
            childrenData = []
            for child_name, child_data in person.childrenData.items():
                gender = widgets.Text(value=child_data['gender'], description='Gender:', disabled=True)
                birthDayChild = widgets.Text(value=child_data['birthDay'], description='BirthDay:', disabled=True)
                childrenData.append(widgets.VBox([widgets.Label(value=child_name), gender, birthDayChild]))

        imageWidgets = []
        for image_data in person.photoData:
            urlImg = image_data[0]["image_path"]
            emotionsDict = image_data[0]["emotions"]
            locationFace = image_data[0]["Location"]

            img = Image.open(urlImg)
            imgWidget = widgets.Image(value=img._repr_png_(), format='png', width=300, height=300)
            emotions = widgets.VBox([widgets.Label(value="Emotions:")] +
                                    [widgets.Text(value=f"{e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%")
                                     for e in emotionsDict.keys()])
            location = widgets.Text(value=str(locationFace), description='Location face:', disabled=True)
            imageWidgets.append(widgets.VBox([imgWidget, emotions, location]))

        display(widgets.VBox([
            widgets.Label(value="DATA:"),
            name,
            id,
            birthDay,
            age,
            nacionality,
            status,
            cellphoneLabel,
            *cellphoneData,
            emailLabel,
            *emailData,
            address,
            civilStatus,
            *childrenData,
            widgets.Label(value="Images Captured: ------------------------------------------------------"),
            *imageWidgets
        ]))

        print("\n")

    root = None
    for person in persons:
        if root is None:
            root = BinaryTreeNode(person)
        else:
            root = insert_node(root, person)

    # Buscar la persona por nombre
    found_person = None

    def search_person(node):
        nonlocal found_person
        if node is not None:
            search_person(node.left)
            if node.person.name == name:
                found_person = node.person
            search_person(node.right)

    search_person(root)

    if found_person:
        display_person_data(found_person)
    else:
        print("Persona no encontrada en el árbol.")

###**7)**

In [84]:
def display_person_with_highest_happiness(persons):
    class BinaryTreeNode:
        def __init__(self, person):
            self.person = person
            self.left = None
            self.right = None

    def build_binary_tree_by_age(persons):
        root = None

        def insert_node(root, person):
            if root is None:
                return BinaryTreeNode(person)

            if person.calcularEdad() < root.person.calcularEdad():
                root.left = insert_node(root.left, person)
            else:
                root.right = insert_node(root.right, person)

            return root

        for person in persons:
            if root is None:
                root = BinaryTreeNode(person)
            else:
                root = insert_node(root, person)

        return root

    def calculate_happiness_average(emotionsList):
        # Calcula el promedio de la emoción de felicidad
        total = 0
        for emotions in emotionsList:
            index = 0
            for x in emotions:
                happiness = x["emotions"]["happiness"]
                index += 1
                total += happiness
        promedio = (total / len(emotionsList)) * 100
        return promedio

    def print_binary_tree_in_order(root):
        output = []

        def traverse(node):
            nonlocal output
            if node is not None:
                traverse(node.left)
                output.append(node.person)
                traverse(node.right)

        traverse(root)
        return output

    def display_person_data(person):
        name = widgets.Text(value=person.name, description='Name:', disabled=True)
        id = widgets.Text(value=person.id, description='ID:', disabled=True)
        birthDay = widgets.Text(value=person.birthDay, description='BirthDay:', disabled=True)
        age = widgets.Text(value=str(person.calcularEdad()), description='Age (In years old):', disabled=True)
        nacionality = widgets.Text(value=person.nacionality, description='Nacionality:', disabled=True)
        status = widgets.Text(value='Hired' if person.accepted else 'Not hired', description='Status:', disabled=True)

        cellphoneLabel = widgets.Label(value="Cellphone:")
        cellphoneData = []
        for x in person.cellphones:
            number = str(x)
            data = widgets.Text(value=f"{number}: {person.cellphones[x]}", disabled=True)
            cellphoneData.append(data)


        emailLabel = widgets.Label(value="Emails:")
        emailData = []
        for email in person.emails:
            emailData.append(widgets.Text(value=email, disabled=True))

        address = widgets.Text(value=person.address, description='Location:', disabled=True)
        civilStatus = widgets.Text(value=person.civilStatus, description='Civil Status:', disabled=True)

        if person.childrenData == {}:
            childrenData = [widgets.Text(value="Dont have children")]
        else:
            childrenData = []
            for child_name, child_data in person.childrenData.items():
                gender = widgets.Text(value=child_data['gender'], description='Gender:', disabled=True)
                birthDayChild = widgets.Text(value=child_data['birthDay'], description='BirthDay:', disabled=True)
                childrenData.append(widgets.VBox([widgets.Label(value=child_name), gender, birthDayChild]))

        imageWidgets = []
        for image_data in person.photoData:
            urlImg = image_data[0]["image_path"]
            emotionsDict = image_data[0]["emotions"]
            locationFace = image_data[0]["Location"]

            img = Image.open(urlImg)
            imgWidget = widgets.Image(value=img._repr_png_(), format='png', width=300, height=300)
            emotions = widgets.VBox([widgets.Label(value="Emotions:")] +
                                    [widgets.Text(value=f"{e}: {'{:.2f}'.format(emotionsDict[e] * 100)}%")
                                     for e in emotionsDict.keys()])
            location = widgets.Text(value=str(locationFace), description='Location face:', disabled=True)
            imageWidgets.append(widgets.VBox([imgWidget, emotions, location]))

        display(widgets.VBox([
            widgets.Label(value="DATA:"),
            name,
            id,
            birthDay,
            age,
            nacionality,
            status,
            cellphoneLabel,
            *cellphoneData,
            emailLabel,
            *emailData,
            address,
            civilStatus,
            *childrenData,
            widgets.Label(value="Images Captured: ------------------------------------------------------"),
            *imageWidgets
        ]))

        print("\n")
    tree_root = build_binary_tree_by_age(persons)
    tree_data = print_binary_tree_in_order(tree_root)

    # Encontrar la persona con el grado de felicidad más alto
    highest_happiness_person = None
    highest_happiness = 0

    for person in tree_data:
        happiness_average = calculate_happiness_average(person.photoData)
        if happiness_average > highest_happiness:
            highest_happiness = happiness_average
            highest_happiness_person = person

    # Mostrar la persona con el grado de felicidad más alto
    if highest_happiness_person:
        display_person_data(highest_happiness_person)
    else:
        print("No se encontró ninguna persona en el árbol.")

# "La programación es una habilidad que puedes cultivar en ti mismo y usar para cambiar el mundo". - Barack Obama

#Main:

In [107]:
def main(b):
 clear_output()
 outputMenu = widgets.Output()
 with outputMenu:

  dataPersons = widgets.Label(value="Show data by: ")
  showData = widgets.Label(value="Persons get data: ")

  buttonInterviewData = widgets.Button(
    description='Interview',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix),
    )
  buttonInterviewData.style.button_color = 'green'

  buttonEvaluatorData = widgets.Button(
    description='Evaluator',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  buttonEvaluatorData.style.button_color = 'green'

  buttonInterviewerData = widgets.Button(
    description='Interviewer',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  buttonInterviewerData.style.button_color = 'green'

  buttonSentences = widgets.Button(
    description='Sentences',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  buttonSentences.style.button_color = 'green'

  buttonBinaryTree = widgets.Button(
    description='Binary Trees',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  buttonBinaryTree.style.button_color = 'green'

  # TwoByTwoLayout(top_left=buttonInterviewerData,
  #              bottom_left=buttonEvaluatorData,
  #              bottom_right=buttonInterviewData)
  # display(TwoByTwoLayout)
  display(widgets.VBox([dataPersons, buttonInterviewData, buttonEvaluatorData, buttonInterviewerData, showData]))
  display(widgets.HBox([buttonSentences, buttonBinaryTree]))   
 display(outputMenu)
 #Cases
 buttonInterviewData.on_click(asignInterviewed)
 buttonEvaluatorData.on_click(evaluatorSave)
 buttonInterviewerData.on_click(interviewerGettingData)

 buttonSentences.on_click(showSentencesButton)
 buttonBinaryTree.on_click(showSentencesButtonBT)

main("")


Output()

## Aspectos administrativos


* La tarea será desarrollada en equipos de trabajo de máximo 2 personas.
* Cualquier acción de plagio será sancionada con una nota de 0 y la carta al expediente respectiva para todos los
implicados.
* La fecha de entrega será la semana 17, el día de clases, antes de la hora de clases.
* Para llevar el control de versiones del proyecto se debe crear un repositorio en GitLab y agregar al profesor (agregarlo
a la cuenta amendez.tec@gmail.com).
* Para llevar el control del las tareas a realizar por participante en el proyecto debe utilizar la herramienta
https://trello.com/es y compartir el link en el mismo Google Doc en que se comparte el link de la plantilla de Overleaf. Pueden crear tab de todas las tareas o requerimientos, un tab para las tareas en proceso y un tab para las tareas hechas.
* La codificación y documentación interna deberá ser desarrollada en idioma inglés utilizando el estándar docString de Python.
*La documentación externa debe estar redactada en idioma inglés utilizando la herramienta Overleaf (utilizar la plantilla facilitada por el profesor). La documentación es evaluada en el curso de Introducción a la programación.
* Documentación del Código utilizando Docstring Conventions 	
* Introducción a Python la Guía de Estilo de Código en Python
* La entrega será mediante la plataforma TecDigital en el espacio de Proyecto – Etapa 2. Lo que debe subir es un archivo de tipo notebook cuya extensión es (.ipynb). Además, en este archivo deben venir los enlaces a las imagenes utilizados para la tarea.
* Se recomienda que se empiece a trabajar lo antes posible.



**URL carpeta de imagenes:** [Directory](https://drive.google.com/drive/folders/1z6mstdNxnfTp0pfmYSUAriuCHDIqS3oO?usp=sharing)